# So You Wanna Be a Pandas Expert?

See README.md for more on setup.
Recommended at least `Python 3.8` because of `:=` walrus operator usage.
All code should run with `Python 3.9`, tested on `3.10`

[Talk by James Powell](https://www.youtube.com/watch?v=pjq3QOxl9Ok)

Notebook follow along / notes from presentation and code snippets

**NOT** A word for word text transcription

Some examples changed slightly or added in

Formatting varies between James' `print(x,y,z, sep='\n')` format and ipython cell outputs.

Other Recommended James Powell Python talks:

- [So you want to be a Python expert?](https://www.youtube.com/watch?v=cKPlPJyQrt4)
- [Objectionable Content](https://www.youtube.com/watch?v=1SHi1kriJI4)

## Code Imports


In [1]:
# Builtin libraries
from random import seed
from random import randint, choice
from string import ascii_lowercase
from collections import Counter
from subprocess import run, check_output
from pathlib import Path
from io import StringIO
from functools import reduce
from operator import and_
from sys import settrace, gettrace
from contextlib import contextmanager
from inspect import signature
from warnings import catch_warnings, filterwarnings
from datetime import timedelta

seed(0)


In [2]:
# Load pandas and numpy and display the versions
import pandas as pd
import numpy as np
from numpy.random import default_rng

# set up re-used rng object
rng = default_rng(0)

"pd:", pd.__version__, "np:", np.__version__


('pd:', '1.4.0', 'np:', '1.22.2')

In [3]:
# Ignore if preferred. Scipy and Xarray examples
import scipy
from scipy.stats import kurtosis, skew, zscore
import xarray
from xarray import DataArray

"scipy:", scipy.__version__, "xarray:", xarray.__version__


('scipy:', '1.8.0', 'xarray:', '0.21.1')

## Intro

Pandas is a very large library and part of larger ecosystem.
Memorizing documentation isn't feasible.

Discover and Understand Core Concept of Pandas API

Core Concept: **Index** and **Index Alignment**

What is the Index?

Why is it so important?

## Is Pandas for Container Types Less Convenient and More Perplexing Than `list` and `dict`

Before understanding the usefulness of `pandas`.
These examples show `Series` and `DataFrame` comparisons to common uses of `list` and `dict`

### Pandas Implementations


In [4]:
# Pandas allows us to create a `Series` with an iterable, just as with a list
s = pd.Series(rng.integers(-10, +10, size=5))
s


0    7
1    2
2    0
3   -5
4   -4
dtype: int64

In [5]:
# Series allows us to iterate over each row
for x in s.iteritems():
    print(f"{x = }")


x = (0, 7)
x = (1, 2)
x = (2, 0)
x = (3, -5)
x = (4, -4)


In [6]:
# Or unpack that those rows like enumerate
for i, x in s.iteritems():
    print(f"{i = }, {x = }")


i = 0, x = 7
i = 1, x = 2
i = 2, x = 0
i = 3, x = -5
i = 4, x = -4


In [7]:
# Or iterate like normal python lists
for x in s:
    print(f"{x = }")


x = 7
x = 2
x = 0
x = -5
x = -4


In [8]:
# DataFrame allows us to create data structures with more columns using iterables
df = pd.DataFrame(rng.integers(-10, +10, size=(5, 3)), columns=[*"abc"])
df


,a,b,c
0,-10,-9,-10
1,-7,6,2
2,8,0,2
3,9,4,2
4,0,1,8


In [9]:
# Iterating over it directly yields columns
for x in df:
    print(f"{x = }")


x = 'a'
x = 'b'
x = 'c'


In [10]:
# iterrows gives some things...
for x in df.iterrows():
    print(f"{type(x)} {x = }")


<class 'tuple'> x = (0, a   -10
b    -9
c   -10
Name: 0, dtype: int64)
<class 'tuple'> x = (1, a   -7
b    6
c    2
Name: 1, dtype: int64)
<class 'tuple'> x = (2, a    8
b    0
c    2
Name: 2, dtype: int64)
<class 'tuple'> x = (3, a    9
b    4
c    2
Name: 3, dtype: int64)
<class 'tuple'> x = (4, a    0
b    1
c    8
Name: 4, dtype: int64)


In [11]:
# iteritems gives some column names and values
for x in df.iteritems():
    print(f"{x = }")


x = ('a', 0   -10
1    -7
2     8
3     9
4     0
Name: a, dtype: int64)
x = ('b', 0   -9
1    6
2    0
3    4
4    1
Name: b, dtype: int64)
x = ('c', 0   -10
1     2
2     2
3     2
4     8
Name: c, dtype: int64)


In [12]:
# Which column names can be unpacked
for col, x in df.iteritems():
    print(f"{x = }\n{col = }")


x = 0   -10
1    -7
2     8
3     9
4     0
Name: a, dtype: int64
col = 'a'
x = 0   -9
1    6
2    0
3    4
4    1
Name: b, dtype: int64
col = 'b'
x = 0   -10
1     2
2     2
3     2
4     8
Name: c, dtype: int64
col = 'c'


In [13]:
# Or get a pretty familiar tuple iteration
for x in df.itertuples():
    print(f"{x = }")


x = Pandas(Index=0, a=-10, b=-9, c=-10)
x = Pandas(Index=1, a=-7, b=6, c=2)
x = Pandas(Index=2, a=8, b=0, c=2)
x = Pandas(Index=3, a=9, b=4, c=2)
x = Pandas(Index=4, a=0, b=1, c=8)


### Python Implementations


In [14]:
# No third-party imports, built in comprehension syntax
some_list = [randint(-10, +10) for _ in range(20)]
some_list


[2, 3, -9, -2, 6, 5, 2, -1, 5, 1, 8, -4, 6, -6, -1, -6, -7, 9, -2, 7]

In [15]:
# Iteration is over the things in the list
for x in some_list:
    print(f"{x = }")


x = 2
x = 3
x = -9
x = -2
x = 6
x = 5
x = 2
x = -1
x = 5
x = 1
x = 8
x = -4
x = 6
x = -6
x = -1
x = -6
x = -7
x = 9
x = -2
x = 7


In [16]:
# Iteration can be modified with builtins such as `enumerate()` if needed
for i, x in enumerate(some_list):
    print(f"{i = }, {x = }")


i = 0, x = 2
i = 1, x = 3
i = 2, x = -9
i = 3, x = -2
i = 4, x = 6
i = 5, x = 5
i = 6, x = 2
i = 7, x = -1
i = 8, x = 5
i = 9, x = 1
i = 10, x = 8
i = 11, x = -4
i = 12, x = 6
i = 13, x = -6
i = 14, x = -1
i = 15, x = -6
i = 16, x = -7
i = 17, x = 9
i = 18, x = -2
i = 19, x = 7


In [17]:
# Dicts are a key-value store and have built in syntax for comprehension  with `{}` and `:`
d = {choice(ascii_lowercase): randint(-10, +10) for _ in range(10)}
d


{'w': 9, 'e': -1, 'd': 1, 'v': 0, 'p': 4, 'n': 0, 't': 10, 'g': 7}

In [18]:
# iterating the dictionary gives the keys / indices
for k in d:
    print(f"{k = }")


k = 'w'
k = 'e'
k = 'd'
k = 'v'
k = 'p'
k = 'n'
k = 't'
k = 'g'


In [19]:
# iteration can be done specifically on keys, values, and items with those methods
for key in d.keys():
    print(f"{key = }")

for value in d.values():
    print(f"{value = }")

for key, value in d.items():
    print(f"{key = }, {value = }")


key = 'w'
key = 'e'
key = 'd'
key = 'v'
key = 'p'
key = 'n'
key = 't'
key = 'g'
value = 9
value = -1
value = 1
value = 0
value = 4
value = 0
value = 10
value = 7
key = 'w', value = 9
key = 'e', value = -1
key = 'd', value = 1
key = 'v', value = 0
key = 'p', value = 4
key = 'n', value = 0
key = 't', value = 10
key = 'g', value = 7


## Is Pandas for the Bizarre Errors

Anyone who writes code will run into Errors.

Pandas reports very specific errors that are hard to understand until you understand what the commands are trying to do to the Index(es)

### Pandas Same Data Shape Makes NaN


In [20]:
s = pd.Series(rng.integers(-10, +10, size=5))
df1 = pd.DataFrame(rng.integers(-10, +10, size=(5, 3)))
# Similar data, setting the index
df2 = pd.DataFrame(
    index=(idx := [1, 1, 2, 3, 4]), data=rng.integers(-10, +10, size=(5, 3))
)
# Similar data, setting a Multi index
df3 = pd.DataFrame(
    index=(idx := pd.MultiIndex.from_product([[0], range(5)])),
    data=rng.integers(-10, +10, size=(5, 3)),
)

# some numeric values in the Series
s


0    -5
1     6
2     3
3   -10
4    -3
dtype: int64

In [21]:
# Some sequences of numerics in the df1 columns
df1


,0,1,2
0,7,1,-10
1,5,4,6
2,-7,-9,7
3,-10,0,-9
4,-5,-1,-2


In [22]:
# try multiplying, maybe it will go across? Nan? maybe need to do other order?
s * df1


,0,1,2,3,4
0,-35,6,-30,NaN,NaN
1,-25,24,18,NaN,NaN
2,35,-54,21,NaN,NaN
3,50,0,-27,NaN,NaN
4,25,-6,-6,NaN,NaN


In [23]:
# Nope, same thing
df1 * s


,0,1,2,3,4
0,-35,6,-30,NaN,NaN
1,-25,24,18,NaN,NaN
2,35,-54,21,NaN,NaN
3,50,0,-27,NaN,NaN
4,25,-6,-6,NaN,NaN


In [24]:
print(df1)
print(df2)


    0  1   2
0   7  1 -10
1   5  4   6
2  -7 -9   7
3 -10  0  -9
4  -5 -1  -2
   0   1   2
1 -2 -10 -10
1 -8 -10   3
2  0   2  -5
3  2   5  -3
4 -1   9   6


In [25]:
# Same data shape, but still NaN
df1 + df2


,0,1,2
0,NaN,NaN,NaN
1,3.0,-6.0,-4.0
1,-3.0,-6.0,9.0
2,-7.0,-7.0,2.0
3,-8.0,5.0,-12.0
4,-6.0,8.0,4.0


In [26]:
# One option is to just pop those off...
(df1 + df2).dropna()


,0,1,2
1,3.0,-6.0,-4.0
1,-3.0,-6.0,9.0
2,-7.0,-7.0,2.0
3,-8.0,5.0,-12.0
4,-6.0,8.0,4.0


In [27]:
# Same with join... some NaN, but we get all our df2 data at least
df1.join(df2, rsuffix="-df2")


,0,1,2,0-df2,1-df2,2-df2
0,7,1,-10,NaN,NaN,NaN
1,5,4,6,-2.0,-10.0,-10.0
1,5,4,6,-8.0,-10.0,3.0
2,-7,-9,7,0.0,2.0,-5.0
3,-10,0,-9,2.0,5.0,-3.0
4,-5,-1,-2,-1.0,9.0,6.0


In [28]:
df3


0  1  2
0 0  9 -3  3
  1  9  3  6
  2  3  4 -3
  3  7 -8  1
  4  4  6  0

In [29]:
# WTF?? It's the same damn shape
df1.join(df3, rsuffix="-df3")


ValueError: cannot join with no overlapping index names

In [ ]:
# Very Smart Coworker (TM) tells you: "oh, you just need to rename the axis"
df1.rename_axis("idx").join(df3.rename_axis([..., "idx"]), rsuffix="-df3")


0  1  2  0-df3  1-df3  2-df3
Ellipsis idx                              
0        0    7 -9  8      7     -9     -9
         1    0 -3  3      3     -4      1
         2    1 -5 -4     -7      7     -1
         3    4  1  0      7      5      4
         4   -4  5 -3     -6      5     -9

### Python builtins have Straightforward Operations


In [ ]:
# 2 Lists with list comprehension
xs = [randint(-10, 10) for _ in range(5)]
ys = [randint(-10, 10) for _ in range(5)]

xs


[-7, 7, 0, 7, -4]

In [ ]:
ys


[9, 7, 8, -1, 4]

In [ ]:
# Python lists aren't mathematical, so '+' is treated as concatenation
xs + ys


[-7, 7, 0, 7, -4, 9, 7, 8, -1, 4]

In [ ]:
# Same concatenation with unpacking syntax
[*xs, *ys]


[-7, 7, 0, 7, -4, 9, 7, 8, -1, 4]

In [ ]:
# Ability to explicitly iterate over each thing and do something with it
[x + y for x, y in zip(xs, ys)]


[2, 14, 8, 6, 0]

In [ ]:
# Dict comprehension to map random letters to random integers
d1 = {choice(ascii_lowercase): randint(-10, 10) for _ in range(10)}
d2 = {choice(ascii_lowercase): randint(-10, 10) for _ in range(10)}

d1


{'c': -6, 'z': 2, 'k': 8, 'h': -1, 'f': -9, 't': -2, 'p': -8, 'e': -9}

In [ ]:
d2


{'c': 7, 'v': 2, 'w': 1, 'i': 4, 'z': -3, 'g': 8, 'n': 8, 'p': 10}

In [ ]:
# Unpacking syntax will "merge" the dicts, with the latter's values taking precedence if there are overlapping keys
# (that's a design choice you don't have to agree with)
{**d1, **d2}


{'c': 7,
 'z': -3,
 'k': 8,
 'h': -1,
 'f': -9,
 't': -2,
 'p': 10,
 'e': -9,
 'v': 2,
 'w': 1,
 'i': 4,
 'g': 8,
 'n': 8}

In [ ]:
# Same with pipe operator (3.9+)
d1 | d2


{'c': 7,
 'z': -3,
 'k': 8,
 'h': -1,
 'f': -9,
 't': -2,
 'p': 10,
 'e': -9,
 'v': 2,
 'w': 1,
 'i': 4,
 'g': 8,
 'n': 8}

In [ ]:
# Ability to manipulate each thing in the iteration
# Unpacking keys into sets
{key: d1.get(key, 0) + d2.get(key, 0) for key in {*d1} | {*d2}}


{'o': -8,
 'b': 7,
 'd': -1,
 'p': 0,
 'g': 8,
 'c': 0,
 'h': -3,
 'z': 7,
 'a': 9,
 'w': -15,
 'x': 2,
 'q': 5,
 'r': -4}

In [ ]:
# Counters give a merge-able and summable key-value / indexed mapping
# Note: outputs exclude negative values
c1 = Counter({choice(ascii_lowercase): randint(-10, 20) for _ in range(10)})
c2 = Counter({choice(ascii_lowercase): randint(-10, 20) for _ in range(10)})

c1


Counter({'n': 14,
         'v': 12,
         'b': -5,
         'o': -8,
         'i': 12,
         'f': 4,
         'q': 18,
         'p': 19,
         'r': 9,
         'y': -10})

In [ ]:
c2


Counter({'b': 5,
         'k': -1,
         'o': -9,
         'z': 3,
         'g': 7,
         'u': -8,
         'x': -6,
         'a': 2,
         'v': 3})

In [ ]:
# Adding counters adds the values of corresponding keys (c1[key] + c2[key])
# Same for subtraction
c1 + c2


Counter({'n': 14,
         'v': 15,
         'i': 12,
         'f': 4,
         'q': 18,
         'p': 19,
         'r': 9,
         'z': 3,
         'g': 7,
         'a': 2})

In [ ]:
# Intersection, keeps min value of matching keys
c1 & c2


Counter({'v': 3})

In [ ]:
# Union, keeps the max value of matching keys
c1 | c2


Counter({'n': 14,
         'v': 12,
         'b': 5,
         'i': 12,
         'f': 4,
         'q': 18,
         'p': 19,
         'r': 9,
         'z': 3,
         'g': 7,
         'a': 2})

## Is Pandas for using `.values` to force it to do what we want

`.values` will often get you a numpy `ndarray` or similar list of values to operate on without using Pandas Indexes

### Pandas to Numpy ndarray


In [ ]:
s = pd.Series(rng.integers(-10, +10, size=5))
df1 = pd.DataFrame(rng.integers(-10, 10, size=(5, 3)), columns=[*"abc"])
# Same df data shape, different column names
df2 = pd.DataFrame(rng.integers(-10, 10, size=(5, 3)), columns=[*"def"])

s


0    1
1   -2
2    9
3   -7
4    8
dtype: int64

In [ ]:
df1


,a,b,c
0,-9,2,1
1,7,-5,8
2,3,7,-7
3,5,8,-10
4,-3,2,-8


In [ ]:
# Well SH*T, dropna isn't going to help us here. I wanted to multiply a vector, right?
s * df1


,a,b,c,0,1,2,3,4
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# But they're the same size... everything is gone....
df1 + df2


,a,b,c,d,e,f
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Back to numpy, works like a champ
df1.values + df2.values


array([[ -9,   4,   6],
       [ 15,  -7,   6],
       [  2,  16, -14],
       [  4,  -2, -12],
       [  5,   4, -12]])

In [ ]:
# Numpy isn't super simple either
s.values + df1.values


ValueError: operands could not be broadcast together with shapes (5,) (5,3) 

In [ ]:
# Smart Coworker (tm) Says: "Oh, you just have to use a new axis to satisfy the broadcast".........
s.values[:, None] + df1.values


array([[  2,  -4, -15],
       [ 11,  10,  12],
       [ -4,  -6,  10],
       [-20, -10, -19],
       [ -8,  -4,  -5]])

In [ ]:
# AND then back to a DataFrame because pandas, right...
pd.DataFrame(df1.values + df2.values)


,0,1,2
0,5,-9,-20
1,-3,-6,9
2,-7,-7,2
3,-8,5,-12
4,-6,8,4


## Is Pandas for the perplexing results and having to just reset the index

Using `reset_index()` to coerce it into doing what we actually want

### Pandas Index Unpredictability


In [ ]:
df1 = pd.DataFrame(rng.integers(-10, 10, size=(5, 3)), columns=[*"abc"])
df2 = pd.DataFrame(rng.integers(-10, 10, size=(5, 3)), columns=[*"abc"])

df1


,a,b,c
0,-3,-4,-2
1,-1,4,7
2,-9,8,0
3,-3,3,1
4,-5,-4,4


In [ ]:
df1.groupby("a").sum()


,b,c
a,,
-9,8,0
-5,-4,4
-3,-1,-1
-1,4,7


In [ ]:
df2


,a,b,c
0,1,0,-4
1,5,-3,-4
2,7,-5,-6
3,4,2,-10
4,-9,-3,6


In [ ]:
# And back to nan-ville
df1.groupby("a").sum() + df2


,a,b,c
-9,NaN,NaN,NaN
-5,NaN,NaN,NaN
-3,NaN,NaN,NaN
-1,NaN,NaN,NaN
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [ ]:
# Yeah, f*ck this index thing, close enough. Maybe groupby can ignore setting index
df1.groupby("a").sum().reset_index() + df2


,a,b,c
0,-8.0,8.0,-4.0
1,0.0,-7.0,0.0
2,4.0,-6.0,-7.0
3,3.0,6.0,-3.0
4,NaN,NaN,NaN


### How Many Rows Will We Even Get?

Indexes seem to mess with row counts for no good reason


In [ ]:
# 2 Series with same Index
s1 = pd.Series(rng.integers(-10, +10, size=4), index=[*"aabb"])
s2 = pd.Series(rng.integers(-10, +10, size=4), index=[*"aabb"])
print(s1, s2, s1 + s2, sep="\n")


a    8
a   -9
b    2
b    1
dtype: int64
a    7
a   -5
b    8
b    3
dtype: int64
a    15
a   -14
b    10
b     4
dtype: int64


In [ ]:
# Series with 2 different Indexes
s1 = pd.Series(
    rng.integers(-10, +10, size=4), index=[*"aaab"]
)  # NOTE different index chars
s2 = pd.Series(rng.integers(-10, +10, size=4), index=[*"abbb"])
print(s1, s2, s1 + s2, sep="\n")


a    7
a   -7
a    5
b    8
dtype: int64
a   -10
b    -3
b     2
b    -8
dtype: int64
a    -3
a   -17
a    -5
b     5
b    10
b     0
dtype: int64


In [ ]:
# Different Indexes in different way
s1 = pd.Series(
    rng.integers(-10, +10, size=4), index=[*"aabb"]
)  # NOTE different index chars
s2 = pd.Series(rng.integers(-10, +10, size=4), index=[*"abbb"])
print(s1, s2, s1 + s2, sep="\n")


a    0
a    2
b    5
b    8
dtype: int64
a   -2
b   -2
b   -1
b    9
dtype: int64
a    -2
a     0
b     3
b     4
b    14
b     6
b     7
b    17
dtype: int64


In [ ]:
# F*ck that pesky index off
s1.reset_index() + s2.reset_index()


,index,0
0,aa,-2
1,ab,0
2,bb,4
3,bb,17


In [ ]:
# Yeah just F*ck it all the way off. LGTM!
s1.reset_index(drop=True) + s2.reset_index(drop=True)


0    -2
1     0
2     4
3    17
dtype: int64

## Is Pandas for the strange API choices such as Groupby

Groupby has multiple ways to pass aggregation functions, and they behave differently with custom user functions


In [ ]:
df = pd.DataFrame(
    {
        "a": np.repeat([True, False], (size := 8) // 2),
        "b": rng.integers(-10, 10, size=size),
    }
)
df


,a,b
0,True,-7
1,True,-1
2,True,-10
3,True,-2
4,False,8
5,False,2
6,False,-4
7,False,9


In [ ]:
df.groupby("a").sum()


,b
a,
False,15
True,-20


In [ ]:
# Doesn't give kurtosis built in, but how
# 8 rows...?
df.groupby("a").transform(lambda x: x.kurt())


,b
0,-3.438272
1,-3.438272
2,-3.438272
3,-3.438272
4,-1.570511
5,-1.570511
6,-1.570511
7,-1.570511


In [ ]:
# Includes a as well?
df.groupby("a").apply(lambda x: x.kurt())


,a,b
a,,
False,0.0,-1.570511
True,0.0,-3.438272


In [ ]:
# Maybe?
df.groupby("a").agg(lambda x: x.kurt())


,b
a,
False,-1.570511
True,-3.438272


## Is Pandas for minor conveniences?

But Numpy + Scipy has it too


In [ ]:
s = pd.Series(rng.integers(-10, +10, size=5))
s


0     2
1     8
2   -10
3    -1
4     6
dtype: int64

In [ ]:
# Can move all elements easily
s.shift(1)


0     NaN
1     2.0
2     8.0
3   -10.0
4    -1.0
dtype: float64

In [ ]:
# Or take the difference between each element and the next
s.diff(1)


0     NaN
1     6.0
2   -18.0
3     9.0
4     7.0
dtype: float64

In [ ]:
# Or do a sum
f"{s.sum() = :.2f}"


's.sum() = 2.00'

In [ ]:
# Or Mean
f"{s.mean() = :.2f}"


's.mean() = 0.40'

In [ ]:
# Or take the skew
f"{s.skew() = :.2f}"


's.skew() = 0.23'

In [ ]:
# Or Kurtosis
f"{s.kurt() = :.2f}"


's.kurt() = 0.84'

### Same Convenience in scipy


In [ ]:
from scipy.stats import kurtosis, skew

# No pd Series wrapper around ndarray
xs = rng.integers(-10, 10, size=5)
xs


array([ 5, -2, -1, -2,  0])

In [ ]:
# normal python slicing
xs[1:]


array([-2, -1, -2,  0])

In [ ]:
xs[1:] - xs[:-1]


array([-7,  1, -1,  2])

In [ ]:
# builtins similar to python and pandas
f"{xs.sum() = :.2f}"


'xs.sum() = 16.00'

In [ ]:
f"{xs.mean() = :.2f}"


'xs.mean() = 3.20'

In [ ]:
# Scipy for more complicated stuff
f"{skew(xs) = :.2f}"


'skew(xs) = -0.20'

In [ ]:
f"{kurtosis(xs) = :.2f}"


'kurtosis(xs) = -1.53'

## Is Pandas for the `DataFrame` at the cost of 250,000 lines of Code Complexity

### DataFrame Convenience


In [ ]:
# DataFrames give group operations on multiple on-dimensional datasets
df = pd.DataFrame(
    {
        "a": pd.Series(rng.integers(-10, +10, size=(size := 5))),
        "b": pd.Series(rng.integers(-10, +10, size=(size := 5))),
        "c": pd.Series(rng.integers(-10, +10, size=(size := 5))),
    }
)
print(df, df.sum(), sep="\n")


   a  b  c
0 -6  4 -7
1  5  4 -8
2 -9  4 -8
3 -2  8  4
4 -5  8  9
a   -17
b    28
c   -10
dtype: int64


In [ ]:
# And do that grouping differently
print(df, df.sum(axis="columns"), sep="\n")


   a  b  c
0 -6  4 -7
1  5  4 -8
2 -9  4 -8
3 -2  8  4
4 -5  8  9
0    -9
1     1
2   -13
3    10
4    12
dtype: int64


In [ ]:
print(df, df.groupby("a").sum(), sep="\n")


   a  b  c
0 -6  4 -7
1  5  4 -8
2 -9  4 -8
3 -2  8  4
4 -5  8  9
    b  c
a       
-9  4 -8
-6  4 -7
-5  8  9
-2  8  4
 5  4 -8


### Checking Pandas Lines of Code


In [ ]:
# Shallow clone recent pandas
d = Path("/tmp/pandas")
if not d.exists():
    run([*"git clone --depth 1 https://github.com/pandas-dev/pandas".split(), d])


In [ ]:
# Get Series that contains all lines of code
# Relies on unix find to find files and wc -l to count lines
s = (
    pd.read_csv(
        StringIO(
            check_output(
                "find pandas -type f -iname *.* -exec wc -l {} ;".split(), cwd=d
            ).decode()
        ),
        delimiter=" ",
        index_col=[1],
        names="lines path".split(),
    )
    .squeeze("columns")
    .pipe(lambda s: s.set_axis(map(Path, s.index)))
)


In [ ]:
s.info()


<class 'pandas.core.series.Series'>
Index: 1877 entries, pandas/arrays/__init__.py to pandas/testing.py
Series name: lines
Non-Null Count  Dtype
--------------  -----
1877 non-null   int64
dtypes: int64(1)
memory usage: 29.3+ KB


In [ ]:
# Each file line count
s


pandas/arrays/__init__.py                               34
pandas/tests/indexing/interval/test_interval.py        175
pandas/tests/indexing/interval/test_interval_new.py    209
pandas/tests/indexing/interval/__init__.py               0
pandas/tests/indexing/test_iat.py                       48
                                                      ... 
pandas/api/types/__init__.py                            23
pandas/api/extensions/__init__.py                       33
pandas/api/indexers/__init__.py                         17
pandas/api/__init__.py                                   6
pandas/testing.py                                       18
Name: lines, Length: 1877, dtype: int64

In [ ]:
# Just non-test files
s.index.map(lambda p: "pandas/_testing" not in str(p) and "pandas/tests" not in str(p))


Index([ True, False, False, False, False, False, False, False, False, False,
       ...
        True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
      dtype='object', length=1877)

In [ ]:
# Use that as mask to select from s
non_tests = s[
    s.index.map(
        lambda p: "pandas/_testing" not in str(p) and "pandas/tests" not in str(p)
    )
]
non_tests


pandas/arrays/__init__.py              34
pandas/errors/__init__.py             238
pandas/conftest.py                   1783
pandas/tseries/api.py                   8
pandas/tseries/offsets.py              83
                                     ... 
pandas/api/types/__init__.py           23
pandas/api/extensions/__init__.py      33
pandas/api/indexers/__init__.py        17
pandas/api/__init__.py                  6
pandas/testing.py                      18
Name: lines, Length: 415, dtype: int64

In [ ]:
# Group into parent folders and file types
per_type_lines = non_tests.pipe(
    lambda s: s.groupby(
        [s.index.map(lambda p: p.parents[1]), s.index.map(lambda p: p.suffix)]
    ).sum()
)
per_type_lines


.                        .py      3088
pandas                   .in      2968
                         .pxd      384
                         .py     70517
                         .pyi     1632
                         .pyx    15380
pandas/_libs             .h        427
                         .pxd      564
                         .py        65
                         .pyi     1017
                         .pyx    19540
pandas/_libs/src         .c       2192
                         .h       2013
pandas/_libs/src/ujson   .c       5345
                         .h        398
pandas/_libs/tslibs/src  .c       1711
                         .h        171
pandas/api               .py        73
pandas/compat            .py       443
pandas/core              .py     88175
pandas/core/_numba       .py       419
pandas/core/arrays       .py      2891
pandas/io                .py     27019
                         .pyx      439
pandas/io/formats        .tpl      261
pandas/plotting          

In [ ]:
# Unstack into full dataframe
per_type_lines.unstack(fill_value=0)


,.c,.h,.in,.pxd,.py,.pyi,.pyx,.tpl
.,0,0,0,0,3088,0,0,0
pandas,0,0,2968,384,70517,1632,15380,0
pandas/_libs,0,427,0,564,65,1017,19540,0
pandas/_libs/src,2192,2013,0,0,0,0,0,0
pandas/_libs/src/ujson,5345,398,0,0,0,0,0,0
pandas/_libs/tslibs/src,1711,171,0,0,0,0,0,0
pandas/api,0,0,0,0,73,0,0,0
pandas/compat,0,0,0,0,443,0,0,0
pandas/core,0,0,0,0,88175,0,0,0
pandas/core/_numba,0,0,0,0,419,0,0,0


In [ ]:
# Sum for lines of each file type
per_type_lines.unstack(fill_value=0).sum()


.c        9248
.h        3009
.in       2968
.pxd       948
.py     198934
.pyi      2649
.pyx     35359
.tpl       261
dtype: int64

In [ ]:
# Sum for ALL lines of code
per_type_lines.unstack(fill_value=0).sum().sum()


253376

### DataFrame Assignment Convenience


In [ ]:
# w/o c seems to work
df = pd.DataFrame(
    {
        "a": pd.Series(rng.integers(-10, +10, size=(size := 5))),
        "b": pd.Series(rng.integers(-10, +10, size=size)),
        # 'c': pd.Series(rng.choice([*ascii_lowercase], size=size)),
    }
)

df["a"][0] *= 10_000
df


,a,b
0,-90000,-7
1,-9,7
2,3,-1
3,-4,7
4,1,5


In [ ]:
# Adding a string column gives us some BS even with 250,000 lines of code
df = pd.DataFrame(
    {
        "a": pd.Series(rng.integers(-10, +10, size=(size := 5))),
        "b": pd.Series(rng.integers(-10, +10, size=size)),
        "c": pd.Series(rng.choice([*ascii_lowercase], size=size)),
    }
)

df["a"][0] *= 10_000
df


/tmp/ipykernel_369/2592172673.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['a'][0] *= 10_000


,a,b,c
0,40000,-2,q
1,-6,9,p
2,5,-7,x
3,-9,8,h
4,1,-9,x


## Why Use Pandas?

### Why Use Numpy

python list is slow
numpy list is fast

python `list`:

- fixed "shape" (linear)
- dynamic size

`numpy.ndarray`:

- fixed size
- dynamic shape

#### Simple Timer Context Manager


In [ ]:
# With simple timer, for sake of non-notebook users
from contextlib import contextmanager
from time import perf_counter, sleep


@contextmanager
def timed(msg):
    start = perf_counter()
    try:
        yield
    finally:
        stop = perf_counter()
        print(
            f"{msg:<24} elapsed \N{mathematical bold capital delta}t: {stop - start:.6f}"
        )


In [ ]:
# Test timer
with timed("sleep"):
    sleep(1)


sleep                    elapsed 𝚫t: 1.001202


#### Make 2 Lists of 100,000


In [ ]:
# Imported with py_ and np_ prefixes for clarity in comparison
from random import randint as py_randint
from numpy.random import randint as np_randint
from numpy import dot as np_dot

SIZE = 100000


In [ ]:
# Time to create a list in python
with timed("Pure Python: CREATE"):
    py_xs = [py_randint(-10, 10) for _ in range(SIZE)]
    py_ys = [py_randint(-10, 10) for _ in range(SIZE)]


Pure Python: CREATE      elapsed 𝚫t: 0.280097


In [ ]:
# Yeah that's better...
with timed("Numpy: CREATE"):
    np_xs = np_randint(-10, 10, size=SIZE)
    np_ys = np_randint(-10, 10, size=SIZE)


Numpy: CREATE            elapsed 𝚫t: 0.004016


In [ ]:
# Writing a list dot product in python multiplies each pair of elements
def py_dot(xs, ys):
    return sum(x * y for x, y in zip(xs, ys))


with timed("Pure Python: DOT"):
    py_dot(py_xs, py_ys)


Pure Python: DOT         elapsed 𝚫t: 0.015208


In [ ]:
# NP definitely wins again
with timed("Numpy: DOT"):
    np_dot(np_xs, np_ys)


Numpy: DOT               elapsed 𝚫t: 0.000820


#### Numpy is Restricted Computation Domain

manager around python class

intermediate between python code and C / FORTRAN.

be unambiguous about these interactions to access closer to raw performance

HAS TO STAY IN NUMPY.
Example using Python dot product to pull out numpy values and apply is slower than pure python


In [ ]:
# Using np array data in pure python calculation and loop
with timed("Numpy Arrays in Pure Python: DOT"):
    py_dot(np_xs, np_ys)


Numpy Arrays in Pure Python: DOT elapsed 𝚫t: 0.048469


#### Numpy is a view of raw memory

This is the fundamental reason for staying in numpy implementation level.

Staying on one side or the other will benefit you.


In [ ]:
# np array wraps around list
xs = np.array([0, 1, 2])
xs


array([0, 1, 2])

In [ ]:
# It has access to raw memory locations
print(xs.__array_interface__["data"][0])
print(f"{xs.__array_interface__['data'][0] = :#_x}")


94608024143968
xs.__array_interface__['data'][0] = 0x560b_a5ac_0860


In [ ]:
# It can do this because it has a strict data type for all elements
f"{xs.dtype = }"


"xs.dtype =dtype('int64')"

In [ ]:
# It knows its size
f"{xs.size = }"


'xs.size = 3'

In [ ]:
# It knows how to jump between memory locations of data elements
f"{xs.strides = }"


'xs.strides = (8,)'

#### Python List is bag of stuff

Python list is not mathematical array / vector, can have multiple types / sub-nesting


In [ ]:
# Adding lists just concats in python
xs = [1, 2, 3]
ys = [4, 5, 6]
xs + ys


[1, 2, 3, 4, 5, 6]

In [ ]:
# Multiply by a scalar repeats the list, doesn't apply it to each element. Because each element might not be multiply-able
xs * 3


[1, 2, 3, 1, 2, 3, 1, 2, 3]

#### Numpy array is mathematical


In [ ]:
# Same thing in numpy adds each element pair
xs = np.array([1, 2, 3])
ys = np.array([4, 5, 6])
xs + ys


array([5, 7, 9])

In [ ]:
xs * 3


array([3, 6, 9])

#### Numpy gives fixed size, higher-dimensional structures

##### Python Lists are actually one-dimensional, even the conventionally called "2d" ones


In [ ]:
# List of lists is kind of like a matrix
xs = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
xs


[[0, 1, 2], [3, 4, 5], [6, 7, 8]]

In [ ]:
# Any coordinate takes 2 coordinates
xs[0][1]


1

In [ ]:
# We can access one row
xs[0]


[0, 1, 2]

In [ ]:
# But there's no native way to access one column without looping. 2 lookups on 2 lists
f"{[row[1] for row in xs] = }"


'[row[1] for row in xs] = [1, 4, 7]'

#### Python lists can be non-uniform


In [ ]:
# Valid python list. No mathematical meaning.
xs = [[0, 1, 2], [3, 4, 5], [6, 7, [8, 9, [10]]]]
xs[0][1]


1

In [ ]:
# Not valid lookup at all levels.
xs[2][2][2][0]


10

In [ ]:
# Numpy can't treat this as an integer array
np.array(xs)


/tmp/ipykernel_371/3767395659.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(xs)


array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, list([8, 9, [10]])]], dtype=object)

#### Python List is not fixed size


In [ ]:
# Convenient indirection between data and what programmer wants to do
xs = [1, 2, 3, 4]
ys = xs
xs


[1, 2, 3, 4]

In [ ]:
# List size changes dynamically. it just works
xs.append(5)
xs.append(6)
xs


[1, 2, 3, 4, 5, 6]

In [ ]:
# Variables that reference same list get updated as well
ys


[1, 2, 3, 4, 5, 6]

#### Numpy doesn't update the array directly


In [ ]:
xs = np.array([0, 1, 2])
ys = xs
xs, ys


(array([0, 1, 2]), array([0, 1, 2]))

In [ ]:
# Append 3 using "horizontal stacking" function. A little clunky, but we can update size of list
xs = np.hstack([xs, [3]])
xs


array([0, 1, 2, 3, 3])

In [ ]:
# BUT, xs couldn't just take the neighboring chunk of memory to resize, it had to make a copy.
# xs no longer points to same memory / array as ys, it points to result of hstack
ys


array([0, 1, 2])

### Why would we use Xarray

#### Real 2d data: Images

columns, rotations, diagonals are all important in image data.

pictures definitely 2 dimensional data.


In [ ]:
# Fake 3x3 image
img = rng.integers(0, 255, size=(3, 3), dtype="uint8")
img


array([[ 94, 129, 193],
       [216, 206, 234],
       [ 14, 162,  32]], dtype=uint8)

In [ ]:
# Can access a row
img[0]


array([ 94, 129, 193], dtype=uint8)

In [ ]:
# Can access in another way
img[0, :]


array([ 94, 129, 193], dtype=uint8)

In [ ]:
# Can access a column
img[:, 0]


array([ 94, 216,  14], dtype=uint8)

#### `xarray` allows adding names to numpy dimensions and selection in a human way

Intended for multi dimensional data, as is numpy


In [ ]:
# Same numpy image, but with dimension names
img = DataArray(data=rng.integers(0, 255, size=(3, 3), dtype="uint8"), dims=[*"xy"])
img


<xarray.DataArray (x: 3, y: 3)>
array([[247, 188,  15],
       [ 68, 183, 231],
       [204,  77, 168]], dtype=uint8)
Dimensions without coordinates: x, y

In [ ]:
# Can pick a row with given x coord
img.sel(x=0)


<xarray.DataArray (y: 3)>
array([247, 188,  15], dtype=uint8)
Dimensions without coordinates: y

In [ ]:
# Or a value with x and y coords
img.sel(x=0, y=1)


<xarray.DataArray ()>
array(188, dtype=uint8)

In [ ]:
# Or selections of different coordinates
img.sel(x=[0, 1], y=[0, 1])


<xarray.DataArray (x: 2, y: 2)>
array([[247, 188],
       [ 68, 183]], dtype=uint8)
Dimensions without coordinates: x, y

##### Can add coordinate grid system


In [ ]:
img = DataArray(
    data=(data := rng.integers(0, 255, size=(3, 3), dtype="uint8")),
    dims=[*"xy"],
    coords={
        "x": np.linspace(10, 20, data.shape[0]),
        "y": np.linspace(10, 20, data.shape[1]),
    },
)
img


<xarray.DataArray (x: 3, y: 3)>
array([[ 28, 239,  65],
       [ 18, 181,  38],
       [ 58,   3,  58]], dtype=uint8)
Coordinates:
  * x        (x) float64 10.0 15.0 20.0
  * y        (y) float64 10.0 15.0 20.0

In [ ]:
# Get values at actual coordinates, not indices
img.sel(x=15, y=15)


<xarray.DataArray ()>
array(181, dtype=uint8)
Coordinates:
    x        float64 15.0
    y        float64 15.0

In [ ]:
# Or try your best to find a close value
img.sel(x=15, y=13, method="nearest")


<xarray.DataArray ()>
array(181, dtype=uint8)
Coordinates:
    x        float64 15.0
    y        float64 15.0

In [ ]:
# Or do an interpolation if the location isn't included
img.interp(x=15, y=range(12, 18), method="linear")


<xarray.DataArray (y: 6)>
array([ 83.2, 115.8, 148.4, 181. , 152.4, 123.8])
Coordinates:
    x        int64 15
  * y        (y) int64 12 13 14 15 16 17

##### X array can add additional axes


In [ ]:
# Scenario where a bunch of readings in 10 different files
img = DataArray(
    data=(data := rng.integers(0, 255, size=(10, 3, 3), dtype="uint8")),
    dims=["filename", *"xy"],
    coords={
        "filename": [Path(f"{x}.bmp") for x in ascii_lowercase[: data.shape[0]]],
        "x": np.linspace(10, 20, data.shape[1]),
        "y": np.linspace(10, 20, data.shape[2]),
    },
)
img


<xarray.DataArray (filename: 10, x: 3, y: 3)>
array([[[119, 121,  49],
        [207,  77,  27],
        [ 63, 165, 120]],

       [[ 88, 169, 232],
        [126, 177, 236],
        [127, 112, 122]],

       [[ 75, 154,  68],
        [153, 129, 247],
        [ 94,  72, 191]],

       [[185, 190, 126],
        [219, 160, 230],
        [  0,  42, 138]],

       [[236, 190,  85],
        [142,   6, 231],
        [ 95, 238,  75]],

       [[ 54,  70,  76],
        [198, 218, 207],
        [175, 143, 189]],

       [[170, 103, 119],
        [178, 239, 244],
        [229,  99, 190]],

       [[215, 125, 218],
        [ 30, 148, 230],
        [140, 114,  15]],

       [[152,   7, 120],
        [210, 206, 194],
        [  6, 178, 201]],

       [[185, 165,  38],
        [184, 215,  72],
        [195, 246,  43]]], dtype=uint8)
Coordinates:
  * filename  (filename) object a.bmp b.bmp c.bmp d.bmp ... h.bmp i.bmp j.bmp
  * x         (x) float64 10.0 15.0 20.0
  * y         (y) float64 10.0 15.0 20.0

In [ ]:
# Select coord from all files
img.sel(x=15, y=15)


<xarray.DataArray (filename: 10)>
array([ 77, 177, 129, 160,   6, 218, 239, 148, 206, 215], dtype=uint8)
Coordinates:
  * filename  (filename) object a.bmp b.bmp c.bmp d.bmp ... h.bmp i.bmp j.bmp
    x         float64 15.0
    y         float64 15.0

In [ ]:
# select from specific file(s)
img.sel(x=15, y=15, filename=Path("a.bmp"))


<xarray.DataArray ()>
array(77, dtype=uint8)
Coordinates:
    filename  object a.bmp
    x         float64 15.0
    y         float64 15.0

## So Why Pandas

Under the covers pandas Series is an array

Small level of indirection from the raw values

With some effor you can get the ndarray underneath

Has some masking mechanism to help with things such as NAN handling

Indexing is a way to refer to values

### PandasArray Indirection


In [ ]:
# This has our raw values underneath. A Masking layer over numpy
s = pd.Series(rng.integers(-10, 10, size=5))
print(s, s.array, sep="\n\n")


0    -5
1    -1
2    -2
3    -2
4   -10
dtype: int64

<PandasArray>
[-5, -1, -2, -2, -10]
Length: 5, dtype: int64


In [ ]:
# Has the ndarray underneath too
s.array._ndarray


array([ -5,  -1,  -2,  -2, -10])

### Pandas index lookup


In [ ]:
# Adding an index doesn't do anything to underlying array
idx = pd.date_range("2000-01-01", periods=5, name="date")
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)
print(s, s.array, sep="\n\n")


date
2000-01-01    2
2000-01-02   -5
2000-01-03    2
2000-01-04    5
2000-01-05   -3
Freq: D, dtype: int64

<PandasArray>
[2, -5, 2, 5, -3]
Length: 5, dtype: int64


In [ ]:
# But it gives us human understanding of each index of the array
s.index


DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05'],
              dtype='datetime64[ns]', name='date', freq='D')

In [ ]:
# Can lookup by date instead of raw matrix index
s["2000-01-01"]


2

In [ ]:
# Or get a range of dates using slices. Works on the other columns of a df if they're all aligned
s["2000-01-02":]


date
2000-01-02   -5
2000-01-03    2
2000-01-04    5
2000-01-05   -3
Freq: D, dtype: int64

### Index for labelling and accessing data

It is a way that we label data in order to access it and give additional meaning to manipulations we might perform

Can be anything relevant to domain:

- some might be numeric values such as levels
  - example will show it can be awkward
- datetime / timestamp for timeseries data
- strings for names and the like


In [ ]:
# Series indexed by some measurements with a step size between them
idx = np.linspace(-20, 20, 5)
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)
print(s, s.index, sep="\n\n")


-20.0   -1
-10.0    9
 0.0     6
 10.0    9
 20.0   -3
dtype: int64

Float64Index([-20.0, -10.0, 0.0, 10.0, 20.0], dtype='float64')


In [ ]:
# Indexing directly with integers gives values at those *labels*
s[0]


6

In [ ]:
# From Label 0 to Label 3, only includes 1
s[0:3]


0.0    6
dtype: int64

In [ ]:
# Accessing by physical memory index instead of labels could be done on the underlying array
print(s.array[0:3], s[0:3], sep="\n\n")
print(f"{len(s.array[0:3]) = }", f"{len(s[0:3]) = }", sep="\n")


<PandasArray>
[-1, 9, 6]
Length: 3, dtype: int64

0.0    6
dtype: int64
len(s.array[0:3]) = 3
len(s[0:3]) = 1


#### Why [], loc[], and iloc[]

Because `loc[]` and `iloc[]` are unambiguous about how the data is accessed.

Raw indexing might use index labels or numeric index if provided with integers


In [ ]:
# Changing dtype of index to int
# Square bracket indexing starts looking like physical indexes
idx = np.linspace(-20, 20, 5, dtype=int)
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)
print(s.array[0:3], s[0:3], sep="\n\n")
print(f"{len(s.array[0:3]) = }", f"{len(s[0:3]) = }", sep="\n")


<PandasArray>
[3, 9, 3]
Length: 3, dtype: int64

-20    3
-10    9
 0     3
dtype: int64
len(s.array[0:3]) = 3
len(s[0:3]) = 3


In [ ]:
# Changing dtype of index to float
# Square bracket indexing starts looking at the labels again?
# But there's a mismatch
idx = np.array([-1, 0, 1, 2, 3, 4], dtype=float)
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)
print(s.array[0:3], s[0:3], sep="\n\n")
print(f"{len(s.array[0:3]) = }", f"{len(s[0:3]) = }", sep="\n")


<PandasArray>
[4, -3, 7]
Length: 3, dtype: int64

0.0   -3
1.0    7
2.0   -8
3.0    1
dtype: int64
len(s.array[0:3]) = 3
len(s[0:3]) = 4


##### In comes Loc and Iloc

Loc is for Labels

Iloc is for Memory Locations

- Returned data is the same as using `.array[]`, just different wrapper


In [ ]:
idx = np.array([-1, 0, 1, 2, 3, 4], dtype=float)
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)
s.loc[0]


0

In [ ]:
s.loc[0:3]


0.0    0
1.0   -3
2.0   -4
3.0   -2
dtype: int64

In [ ]:
s.iloc[0]


6

In [ ]:
s.iloc[0:3]


-1.0    6
 0.0    0
 1.0   -3
dtype: int64

In [ ]:
s.array[0:3]


<PandasArray>
[6, 0, -3]
Length: 3, dtype: int64

In [ ]:
# More discussion ahead
s.iloc[0 : 3 + 1]


-1.0    6
 0.0    0
 1.0   -3
 2.0   -4
dtype: int64

In [ ]:
s.loc[0 : 3 + 1]


0.0    0
1.0   -3
2.0   -4
3.0   -2
4.0   -1
dtype: int64

#### loc including end boundary

Unlike `range()` and slicing, `loc` includes the final integer provided because this method of indexing is not exclusive to integers.

Index labels might be ambigous.

This is a major source of unknown errors.


In [ ]:
# Use letters abcde as index
# Index doesn't have to be numeric
idx = [*"abcde"]
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)
s.loc["a"]


4

In [ ]:
# This slicing makes sense including endpoint
s.loc["a":"c"]


a    4
b    7
c   -9
dtype: int64

In [ ]:
# Because +1 doesn't make sense in all cases
s.loc["a" : "c" + 1]


TypeError: can only concatenate str (not "int") to str

In [ ]:
# Letter index can handle some nonsense by just including the inbetween bits
idx = [*".a...c."]
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)
print(s.loc["a"], s.loc["a":"c"], sep="\n")


3
a    3
.    1
.   -5
.   -4
c    4
dtype: int64


In [ ]:
# Even if its repeated pandas has a reasonable choice to include all
idx = [*".aa.cc."]
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)
print(s.loc["a"], s.loc["a":"c"], sep="\n")


a   -4
a    5
dtype: int64
a   -4
a    5
.   -3
c   -4
c    7
dtype: int64


In [ ]:
# It throws an error when there isn't a reasonable choice
# First a to first c? first a to last c? last a to last c?
idx = [*".ac.ac."]
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)
print(s.loc["a"], s.loc["a":"c"], sep="\n")


KeyError: "Cannot get left slice bound for non-unique label: 'a'"

### Types of Pandas Index

Standard Range index, which has a step

Integer 64 Index, bunch of integers that map to memory

DatetimeIndex, with set periodicity or freq, can change

IntervalIndex, values between discrete intervals

PeriodIndex, such as Quarters


In [ ]:
# RangeIndex has a Pandas/Numpy array underneath
idx = range(0, 5)
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)

print(s.index, s.index.array, sep="\n\n")


RangeIndex(start=0, stop=5, step=1)

<PandasArray>
[0, 1, 2, 3, 4]
Length: 5, dtype: int64


In [ ]:
# RangeIndex can specify step just as in range()
idx = range(0, 5, 2)
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)

print(s.index, s.index.array, sep="\n\n")


RangeIndex(start=0, stop=5, step=2)

<PandasArray>
[0, 2, 4]
Length: 3, dtype: int64


In [ ]:
# Integers not in order is just a bunch of integers mapping to indices
idx = np.arange(0, 5)
rng.shuffle(idx)
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)

print(s.index, s.index.array, sep="\n\n")


Int64Index([2, 0, 1, 3, 4], dtype='int64')

<PandasArray>
[2, 0, 1, 3, 4]
Length: 5, dtype: int64


In [ ]:
# Date ranges go for certain number of periods and have a certain real world frequency
idx = pd.date_range("2000-01-01", periods=5, name="date")
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)

print(s.index, s.index.array, sep="\n\n")


DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05'],
              dtype='datetime64[ns]', name='date', freq='D')

<DatetimeArray>
['2000-01-01 00:00:00', '2000-01-02 00:00:00', '2000-01-03 00:00:00',
 '2000-01-04 00:00:00', '2000-01-05 00:00:00']
Length: 5, dtype: datetime64[ns]


In [ ]:
# Can change the frequency
idx = pd.date_range("2000-01-01", periods=5, freq="2T", name="date")
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)

print(s.index, s.index.array, sep="\n\n")


DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 00:02:00',
               '2000-01-01 00:04:00', '2000-01-01 00:06:00',
               '2000-01-01 00:08:00'],
              dtype='datetime64[ns]', name='date', freq='2T')

<DatetimeArray>
['2000-01-01 00:00:00', '2000-01-01 00:02:00', '2000-01-01 00:04:00',
 '2000-01-01 00:06:00', '2000-01-01 00:08:00']
Length: 5, dtype: datetime64[ns]


In [ ]:
# All intervals in a set start and stop boundary
idx = pd.interval_range(0, 5, name="value")
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)

print(s.index, s.index.array, sep="\n\n")


IntervalIndex([(0, 1], (1, 2], (2, 3], (3, 4], (4, 5]], dtype='interval[int64, right]', name='value')

<IntervalArray>
[(0, 1], (1, 2], (2, 3], (3, 4], (4, 5]]
Length: 5, dtype: interval[int64, right]


In [ ]:
# Configure how the intervals are chosen
idx = pd.interval_range(0, 10, freq=2, closed="left", name="value")
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)

print(s.index, s.index.array, sep="\n\n")


IntervalIndex([[0, 2), [2, 4), [4, 6), [6, 8), [8, 10)], dtype='interval[int64, left]', name='value')

<IntervalArray>
[[0, 2), [2, 4), [4, 6), [6, 8), [8, 10)]
Length: 5, dtype: interval[int64, left]


In [ ]:
# Period range for real world periods
idx = pd.period_range("2000Q1", periods=3, freq="Q", name="quarter")
s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)

print(s.index, s.index.array, sep="\n\n")


PeriodIndex(['2000Q1', '2000Q2', '2000Q3'], dtype='period[Q-DEC]', name='quarter')

<PeriodArray>
['2000Q1', '2000Q2', '2000Q3']
Length: 3, dtype: period[Q-DEC]


### But what is Index

Underneath it is numpy array data

...

But it's not data

#### Index is a "mechanism" to access data

Lazy by nature because it's a description / label, not the thing itself.


In [ ]:
# Normal, do-able in normal python list in memory
idx = pd.RangeIndex(0, 100)
idx


RangeIndex(start=0, stop=100, step=1)

In [ ]:
# No way on earth this many numbers would fit. Index just has to do a bit of arithmetic based on the logic of itself
idx = pd.RangeIndex(0, 100_000_000_000_000)
idx


RangeIndex(start=0, stop=100000000000000, step=1)

In [ ]:
idx = pd.RangeIndex(10, 20, 5)
idx


RangeIndex(start=10, stop=20, step=5)

#### How Index does it

With a common API to data

Features such as checking or uniqueness, whether it is monotonic


In [ ]:
# Index knows mathematical properties about itself
idx.has_duplicates


False

In [ ]:
idx.is_monotonic


True

In [ ]:
idx.is_unique


True

In [ ]:
# Index has a method to translate a key to a physical position
idx.get_loc


<bound method RangeIndex.get_loc of RangeIndex(start=10, stop=20, step=5)>

In [ ]:
# Calling on a label in the Index returns the value pointed to by that label. Gets calculated by the logic within the Index description
# Translate human label to machine location
idx.get_loc(10)


0

In [ ]:
# Translates the human label slice operation to the equivalent slice on memory locations
idx.slice_locs(10, 13)


In [ ]:
# Returns the array of each machine index of the labels 10 through 30 jumping by 5.
# -1 means "not found" in the original index labels --> NaN?
idx.get_indexer(range(10, 30, 5))


array([ 0,  1, -1, -1])

#### All Index Types have Common API


In [ ]:
# Bunch of index types
indices = [
    (range(5), pd.RangeIndex),
    (range(5, 10), pd.RangeIndex),
    (range(5, 15, 2), pd.RangeIndex),
    ([0, 1, 2, 3, 5], pd.Int64Index),
    ([0, 1, 2, 3, 5.0], pd.Float64Index),
    ([*"abcde"], pd.Index),
    (pd.interval_range(0, 10, 5, closed="left"), pd.IntervalIndex),
    (pd.date_range("2000-01-01", periods=5), pd.DatetimeIndex),
    (
        pd.to_datetime("2000-01-01") + pd.to_timedelta([0, 0, 1, 1, 2], unit="D"),
        pd.DatetimeIndex,
    ),
    (pd.period_range("2000Q1", periods=5, freq="Q"), pd.PeriodIndex),
]


/tmp/ipykernel_373/1595331364.py:6: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  ([0, 1, 2, 3, 5], pd.Int64Index),
/tmp/ipykernel_373/1595331364.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  ([0, 1, 2, 3, 5.0], pd.Float64Index),


In [ ]:
# All have the expected types

for idx, typ in indices:
    s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)
    assert isinstance(s.index, typ)


In [ ]:
# All have those self-aware methods

for idx, _ in indices:
    s = pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx)
    print(
        s.index,
        f"{s.index.get_loc(s.index[0]) = }",
        f"{s.index.is_monotonic        = }",
        f"{s.index.is_unique           = }",
        f"{s.index.has_duplicates      = }",
        sep="\n",
        end="\n\n",
    )


RangeIndex(start=0, stop=5, step=1)
s.index.get_loc(s.index[0]) = 0
s.index.is_monotonic        = True
s.index.is_unique           = True
s.index.has_duplicates      = False

RangeIndex(start=5, stop=10, step=1)
s.index.get_loc(s.index[0]) = 0
s.index.is_monotonic        = True
s.index.is_unique           = True
s.index.has_duplicates      = False

RangeIndex(start=5, stop=15, step=2)
s.index.get_loc(s.index[0]) = 0
s.index.is_monotonic        = True
s.index.is_unique           = True
s.index.has_duplicates      = False

Int64Index([0, 1, 2, 3, 5], dtype='int64')
s.index.get_loc(s.index[0]) = 0
s.index.is_monotonic        = True
s.index.is_unique           = True
s.index.has_duplicates      = False

Float64Index([0.0, 1.0, 2.0, 3.0, 5.0], dtype='float64')
s.index.get_loc(s.index[0]) = 0
s.index.is_monotonic        = True
s.index.is_unique           = True
s.index.has_duplicates      = False

Index(['a', 'b', 'c', 'd', 'e'], dtype='object')
s.index.get_loc(s.index[0]) = 0
s.index.is_mon

In [ ]:
# They really have a lot of common (non-__dunder__) methods!
methods = {
    frozenset(
        x
        for x in dir(pd.Series(rng.integers(-10, 10, size=len(idx)), index=idx).index)
        if not x.startswith("__")
    )
    for idx, _ in indices
}
methods


{frozenset({'T',
            '_accessors',
            '_arith_method',
            '_assert_can_do_setop',
            '_assert_safe_casting',
            '_attributes',
            '_cache',
            '_can_hold_identifiers_and_holds_name',
            '_can_hold_na',
            '_can_hold_strings',
            '_can_use_libjoin',
            '_check_indexing_error',
            '_check_indexing_method',
            '_cleanup',
            '_cmp_method',
            '_comparables',
            '_concat',
            '_construct_result',
            '_constructor',
            '_convert_can_do_setop',
            '_convert_slice_indexer',
            '_convert_tolerance',
            '_data',
            '_data_cls',
            '_default_dtype',
            '_deprecated_arg',
            '_deprecation_message',
            '_difference',
            '_difference_compat',
            '_dir_additions',
            '_dir_additions_for_owner',
            '_dir_deletions',
           

In [ ]:
sorted(reduce(and_, methods))


['T',
 '_accessors',
 '_arith_method',
 '_assert_can_do_setop',
 '_attributes',
 '_cache',
 '_can_hold_identifiers_and_holds_name',
 '_can_hold_na',
 '_can_hold_strings',
 '_can_use_libjoin',
 '_check_indexing_error',
 '_check_indexing_method',
 '_cleanup',
 '_cmp_method',
 '_comparables',
 '_concat',
 '_construct_result',
 '_constructor',
 '_convert_can_do_setop',
 '_convert_slice_indexer',
 '_convert_tolerance',
 '_data',
 '_data_cls',
 '_deprecated_arg',
 '_difference',
 '_difference_compat',
 '_dir_additions',
 '_dir_additions_for_owner',
 '_dir_deletions',
 '_drop_level_numbers',
 '_dtype_to_subclass',
 '_duplicated',
 '_engine',
 '_engine_type',
 '_ensure_array',
 '_filter_indexer_tolerance',
 '_find_common_type_compat',
 '_format_attrs',
 '_format_data',
 '_format_duplicate_message',
 '_format_native_types',
 '_format_space',
 '_format_with_header',
 '_formatter_func',
 '_from_join_target',
 '_get_attributes_dict',
 '_get_engine_target',
 '_get_fill_indexer',
 '_get_fill_indexer

In [ ]:
# Most of the first type's methods
len(next(iter(methods))), len(sorted(reduce(and_, methods)))


(261, 242)

### Extending Index

Somewhat difficult to extend in this way

Maybe a symmetric index to represent data that is stored as positives but can be looked up as positive or negative


In [ ]:
class SymmetricIndex(pd.Int64Index):
    """Subclass Int64Index to start with"""

    def get_loc(self, key):
        """Override get_loc with custom logic"""
        return super().get_loc(abs(key))


/tmp/ipykernel_373/4036981236.py:1: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  class SymmetricIndex(pd.Int64Index):


In [ ]:
idx1 = SymmetricIndex([0, 2, 4])
s1 = pd.Series(rng.integers(-10, 10, size=len(idx1)), index=idx1)
s1


0    1
2    4
4   -4
dtype: int64

In [ ]:
s1.index


SymmetricIndex([0, 2, 4], dtype='int64')

In [ ]:
# Get Loc still works on positive
s1.index.get_loc(2)


1

In [ ]:
# and gets abs value for negative
s1.index.get_loc(-2)


1

In [ ]:
# Even the loc indexer works to get value
s1.loc[2]


4

In [ ]:
# Taking negative to positive
s1.loc[-2]


4

In [ ]:
# Introduce another series
idx2 = SymmetricIndex([-4, -3, -2, -1, 0, 1, 2, 3, 4])
s2 = pd.Series(rng.integers(-10, 10, size=len(idx2)), index=idx2)
s2


-4   -4
-3   -3
-2    9
-1    7
 0   -6
 1   -1
 2    1
 3    8
 4   -8
dtype: int64

In [ ]:
# These don't seem to line up...
s1 + s2


-4     NaN
-3     NaN
-2     NaN
-1     NaN
 0    -5.0
 1     NaN
 2     5.0
 3     NaN
 4   -12.0
dtype: float64

### Checking out S1 + S2

Trace how pandas adds 2 series which are timeseries that are slightly offset

677 function calls from `s1 + s2` adding 2 Series

Things with not too much depth include checking / validation of operands


In [ ]:
# Path dependent on your kernel environment install path
@contextmanager
def tracing():
    old_trace = gettrace()
    depth, frames = 0, []

    def trace(frame, event, arg):
        nonlocal depth
        filename = frame.f_code.co_filename
        if filename.startswith("<"):
            return
        filename = Path(filename)
        if not filename.is_relative_to(
            base_dir := Path(
                "~/miniconda3/envs/py310/lib/python3.10/site-packages/pandas"
            ).expanduser()
        ):
            return

        if event == "return":
            depth -= 1
        if event == "call":
            depth += 1
            fr = (
                filename.relative_to(base_dir),
                frame.f_lineno,
                frame.f_code.co_name,
                depth,
            )
            frames.append(fr)
            return trace

    settrace(trace)
    try:
        yield lambda: pd.DataFrame(
            frames, columns="filename lineno function depth".split()
        )
    finally:
        settrace(old_trace)


In [ ]:
idx1 = pd.date_range("2000-01-01", periods=5, name="date")
s1 = pd.Series(rng.integers(-10, 10, size=len(idx1)), index=idx1)

idx2 = pd.date_range("2000-01-02", periods=5, name="date")
s2 = pd.Series(rng.integers(-10, 10, size=len(idx2)), index=idx2)

with tracing() as get_df:
    s1 + s2

df = get_df()
print(s1, s2, sep="\n\n")


date
2000-01-01    -4
2000-01-02    -8
2000-01-03     4
2000-01-04   -10
2000-01-05    -1
Freq: D, dtype: int64

date
2000-01-02    4
2000-01-03    0
2000-01-04    3
2000-01-05    5
2000-01-06   -6
Freq: D, dtype: int64


In [ ]:
# A Lot of function calls on s1 + s2
df


,filename,lineno,function,depth
0,core/ops/common.py,55,new_method,1
1,core/dtypes/generic.py,43,_check,2
2,core/dtypes/generic.py,43,_check,2
3,core/dtypes/generic.py,43,_check,2
4,core/arraylike.py,98,__add__,2
...,...,...,...,...
535,core/series.py,641,name,7
536,core/dtypes/common.py,1721,validate_all_hashable,8
537,core/dtypes/common.py,1740,<genexpr>,9
538,core/dtypes/inference.py,321,is_hashable,10


In [ ]:
# Show traces on shallow calls. It's a lot of checks and things
df[df["depth"] <= 5]


,filename,lineno,function,depth
0,core/ops/common.py,55,new_method,1
1,core/dtypes/generic.py,43,_check,2
2,core/dtypes/generic.py,43,_check,2
3,core/dtypes/generic.py,43,_check,2
4,core/arraylike.py,98,__add__,2
5,core/series.py,5634,_arith_method,3
6,core/ops/__init__.py,152,align_method_SERIES,4
7,core/dtypes/generic.py,43,_check,5
8,core/indexes/datetimelike.py,109,equals,5
23,core/series.py,4493,align,5


In [ ]:
# Over 100 calls into the core index modules
df[df["filename"].apply(lambda x: x.is_relative_to("core/indexes"))]


,filename,lineno,function,depth
8,core/indexes/datetimelike.py,109,equals,5
9,core/indexes/base.py,803,is_,6
10,core/indexes/extension.py,67,fget,6
12,core/indexes/extension.py,67,fget,6
14,core/indexes/extension.py,67,fget,6
...,...,...,...,...
474,core/indexes/base.py,7082,maybe_extract_name,7
478,core/indexes/base.py,6987,ensure_index,7
480,core/indexes/base.py,884,__len__,8
501,core/indexes/base.py,884,__len__,8


### Index as Operational Metadata

Index is well-defined semantics of how data can interact.

Index gets preserved through these Operations as defined by those semantic rules

Some operations such as groupby aren't the best at preserving the Metadata

Order of operations will affect this in some situations, such as addition, where it defaults to taking one Series' metadata.
This is to avoid ambiguity of what to do with the Metadata in this situation:

- Should it merge?
- What would that mean?
- Set, List, Concatenation?
- What about dates?


In [ ]:
# Can attach metadata to Series
s1 = pd.Series(
    data=rng.integers(-10, 10, size=(size := 1_000)),
    index=rng.choice([*ascii_lowercase], size=size),
    name="series1",
)
s1.attrs["source"] = ...
s1.attrs["author"] = ...

print(s1, s1.attrs, sep="\n")


m    6
k    2
v    7
s   -6
n    7
    ..
h    2
b   -4
r   -6
l   -3
b   -4
Name: series1, Length: 10000, dtype: int64
{'source': Ellipsis, 'author': Ellipsis}


In [ ]:
# attrs get preserved
s2 = pd.Series(
    data=rng.integers(-10, 10, size=(size := 1_000)),
    index=rng.choice([*ascii_lowercase], size=size),
    name="series2",
)

s2.attrs["source"] = ...
# This one stores date
s2.attrs["date"] = ...

print(s2, s2.attrs, sep="\n")


l    9
g   -8
x    8
r   -2
b   -5
    ..
b    2
z    1
t   -6
i    4
u    6
Name: series2, Length: 10000, dtype: int64
{'source': Ellipsis, 'date': Ellipsis}


In [ ]:
# Newer pandas will preserve hopefully, older may not
print(
    s1.groupby(s1.index).mean().head(3),
    f"{s1.groupby(s1.index).mean().attrs = }",
    sep="\n\n",
)


a   -0.676166
b   -0.474114
c   -0.620253
Name: series1, dtype: float64

s1.groupby(s1.index).mean().attrs = {'source': Ellipsis, 'author': Ellipsis}


In [ ]:
# metadata gets overridden by latter operands
f"{(lsum := s1 + s2).dropna().attrs = }"


"(lsum := s1 + s2).dropna().attrs = {'source': Ellipsis, 'author': Ellipsis}"

In [ ]:
f"{(rsum := s2 + s1).dropna().attrs = }"


"(rsum := s2 + s1).dropna().attrs = {'source': Ellipsis, 'date': Ellipsis}"

In [ ]:
# Name may be from one side or the other or neither
# `name` property may or may not propogate from one Series or the other in different verions.
# This metadata isn't well defined
f"{lsum.name}", f"{rsum.name}"


('None', 'None')

In [ ]:
# BUT Index operations preserve these!
s2.loc[s1[s1 > 0].index].attrs


{'source': Ellipsis, 'date': Ellipsis}

In [ ]:
s2.loc[s1[s1 > 0].index].name


'series2'

##### Index Has to Be Preserved.

It is the persistent metadata that directly affects the data.

What are the rules of `.index`?
What are the rules of `numpy.ndarray`?
What are the rules of `numpy` promotion? - Loss of precision can occur in pandas - NaN's will promote to float

### What are rules of numpy


In [ ]:
xs = np.array([x := 2 ** 53, x + 1, x + 2, x + 3])
ys = np.exp(np.log(xs))
# 4 values
xs


array([9007199254740992, 9007199254740993, 9007199254740994,
       9007199254740995])

In [ ]:
# When do integer data turn non-integer.
# On exp(log(x)) we get float
ys


array([9.00719925e+15, 9.00719925e+15, 9.00719925e+15, 9.00719925e+15])

In [ ]:
# Checking xs for whether data is distinct, it all is
(xs[0] == xs).all()


False

In [ ]:
np.equal(*np.meshgrid(xs, xs)).all()


False

In [ ]:
np.allclose(*np.meshgrid(xs, xs), atol=1, rtol=0)


False

In [ ]:
# BUT ys has promoted everything to floats and now they're not distinct
# Can't be precisely stored with last digit
(ys[0] == ys).all()


True

In [ ]:
np.equal(*np.meshgrid(ys, ys)).all()


True

In [ ]:
np.allclose(*np.meshgrid(ys, ys), atol=1, rtol=0)


True

#### Numpy Promotion can cause loss of precision


In [ ]:
# Different index labelling
s1 = pd.Series([x := 2 ** 53, x + 1, x + 2, x + 3], dtype=np.int64, index=range(4))
s2 = pd.Series(
    [x := 2 ** 53, x + 1, x + 2, x + 3], dtype=np.int64, index=range(1, 4 + 1)
)
print(s1, s2, sep="\n")


0    9007199254740992
1    9007199254740993
2    9007199254740994
3    9007199254740995
dtype: int64
1    9007199254740992
2    9007199254740993
3    9007199254740994
4    9007199254740995
dtype: int64


In [ ]:
# Same values, makes sense
(s1.values == s2.values).all()


True

In [ ]:
# Index labels don't match, as expected
(s1.index == s2.index).all()


False

In [ ]:
s1 + s2


0             NaN
1    1.801440e+16
2    1.801440e+16
3    1.801440e+16
4             NaN
dtype: float64

In [ ]:
# Adding them gives NaNs, so we had to promote to flaot
(s1 + s2).dtype


dtype('float64')

In [ ]:
# Maybe try add method?
s1.add(s2, fill_value=0)


0    9.007199e+15
1    1.801440e+16
2    1.801440e+16
3    1.801440e+16
4    9.007199e+15
dtype: float64

In [ ]:
s1.add(s2, fill_value=0).dtype


dtype('float64')

#### Numpy Broadcasting


In [ ]:
xs = rng.normal(size=(2, 3, 4)).round(1)
ys = rng.normal(size=2).round(1)

print(
    xs,
    ys,
    sep="\n{}\n".format("\N{box drawings light horizontal}" * 40),
)


[[[-0.2 -1.2 -0.5  0.4]
  [-0.3 -0.4 -0.1  0.9]
  [-1.7  1.5  0.  -1.1]]

 [[-1.1 -1.  -0.2  0.3]
  [ 0.2  0.4  1.6  0.2]
  [ 2.5  1.4 -0.9 -1.5]]]
────────────────────────────────────────
[-2.6 -0.4]


In [ ]:
# Broadcast error? trying to add those 2 values
ys = rng.normal(size=2).round(1)

print(
    xs,
    ys,
    xs + ys,
    sep="\n{}\n".format("\N{box drawings light horizontal}" * 40),
)


ValueError: operands could not be broadcast together with shapes (2,3,4) (2,) 

In [ ]:
# Maybe do it more like xs shape?
ys = rng.normal(size=(2, 3)).round(1)

print(
    xs,
    ys,
    xs + ys,
    sep="\n{}\n".format("\N{box drawings light horizontal}" * 40),
)


ValueError: operands could not be broadcast together with shapes (2,3,4) (2,3) 

In [ ]:
# This one works, but it's not what we wanted
ys = rng.normal(size=(3, 4)).round(1)

print(
    xs,
    ys,
    xs + ys,
    sep="\n{}\n".format("\N{box drawings light horizontal}" * 40),
)


[[[ 0.7  0.5 -0.3  1.1]
  [-0.7 -1.5 -1.2  0.1]
  [-0.4  0.6 -0.3  0.3]]

 [[-0.8  0.7  0.4 -0.1]
  [ 0.1 -1.3 -0.1 -0.4]
  [-0.2  0.5  1.5  0. ]]]
────────────────────────────────────────
[[-1.9  1.5 -0.2 -0.5]
 [-0.7  0.5 -1.8 -0.1]
 [ 1.3  1.1 -1.3  0.4]]
────────────────────────────────────────
[[[-1.2  2.  -0.5  0.6]
  [-1.4 -1.  -3.   0. ]
  [ 0.9  1.7 -1.6  0.7]]

 [[-2.7  2.2  0.2 -0.6]
  [-0.6 -0.8 -1.9 -0.5]
  [ 1.1  1.6  0.2  0.4]]]


In [ ]:
# Smart Coworker (TM): Use newaxis
ys = rng.normal(size=2).round(1)[:, np.newaxis, np.newaxis]

print(
    xs,
    ys,
    xs + ys,
    sep="\n{}\n".format("\N{box drawings light horizontal}" * 40),
)


[[[ 0.7  0.5 -0.3  1.1]
  [-0.7 -1.5 -1.2  0.1]
  [-0.4  0.6 -0.3  0.3]]

 [[-0.8  0.7  0.4 -0.1]
  [ 0.1 -1.3 -0.1 -0.4]
  [-0.2  0.5  1.5  0. ]]]
────────────────────────────────────────
[[[-0.1]]

 [[-1.4]]]
────────────────────────────────────────
[[[ 0.6  0.4 -0.4  1. ]
  [-0.8 -1.6 -1.3  0. ]
  [-0.5  0.5 -0.4  0.2]]

 [[-2.2 -0.7 -1.  -1.5]
  [-1.3 -2.7 -1.5 -1.8]
  [-1.6 -0.9  0.1 -1.4]]]


##### SIDEBAR Rules are not guaranteed to follow intuition

Rules do not necessarily drive conceptual understanding of what you want to do.


In [ ]:
# f-strings are great, but the implementation decision was not to allow \ characters within the expressions
# Nice
"-" * 40


'----------------------------------------'

In [ ]:
# Nicer
"\N{box drawings light horizontal}" * 40


'────────────────────────────────────────'

In [ ]:
# F string nice
f"\n{'-' * 40}\n"


'\n----------------------------------------\n'

In [ ]:
# F string broke
f"\n{'\N{box drawings light horizontal}' * 40}\n"

SyntaxError: f-string expression part cannot include a backslash (631214921.py, line 2)

### Focusing on the "right" rules of broadcasting

can help ndarray broadcasting click

- take dimensionality
- right-align them
- do they match up
  - is there a zip possibility in any column(s)?
  - matching digit or a 1 in one of the entries
- Fill 1's on the right to get broadcast in those dimensions and push match left

Intuition Shape:
xs 2 3 4
ys 2
ys 2 3
ys 3 4
ys 2

Reality comparison (only the 3-4 ys matches any column):
xs 2 3 4
ys 2
ys 2 3
ys 3 4
ys 2

New Axis (nest our data deeper in the layered structure):
xs 2 3 4
ys 2 1 1

The right-most axis is most contiguous in memory

This is why it is last in strides and smallest

Indexing works from left hand side, but broadcasting works from the right

Axis with `newaxis` is just a constant basically; it has a 0 stride


In [ ]:
xs = rng.normal(size=(2, 3, 4)).round(1)
ys = rng.normal(size=2).round(1)[:, np.newaxis, np.newaxis]
zs = rng.normal(size=(3, 4)).round(1)

xs


array([[[ 2.2, -0.8,  1.1,  0.6],
        [-0.9, -0.1, -0.3, -0.6],
        [-0.8,  0.5,  0.6, -0.8]],

       [[ 1.2,  1. , -0.9, -2. ],
        [-0.6,  1.1, -0.4, -2.3],
        [ 0.3, -0.1, -1.7,  0. ]]])

In [ ]:
# Has some kind of shape representing block of memory
xs.shape


(2, 3, 4)

In [ ]:
# Strides relate to traversing that block
# layer to layer, row to row, column to column
xs.strides


(96, 32, 8)

In [ ]:
# And we can go one layer in at a time with indexing
xs[0].strides


(32, 8)

In [ ]:
# A little deeper
xs[0, 1].strides


(8,)

In [ ]:
# Skips more values
xs[:, 1].strides


(96, 8)

In [ ]:
# newaxis adds dimensions for 'free'. No cost to compute
ys.strides


(8, 0, 0)

In [ ]:
# Can figure out how numpy would transform it into something compatible with another shape
np.broadcast_to(zs, xs.shape).strides


(0, 32, 8)

## Rules of Index Alignment

Just need to match up

Without duplicates (simplest)

2 series indexed the same.

- Plus operation works same as counter
  - add up values for corresponding keys
  - Fill in NaN when you can't match a corresponding key

`get_indexer(other_index)` shows what happens when you would do an operation on 2 Series, - Provides some configuration options into how they match including back filling, forward filling, nearest - Reason why Indexes can have different behaviour when passed strings / datetimes / etc.


In [ ]:
# No duplicates makes easy adding / matching
# Kinda like Counter built in
idx1 = [*"abcd"]
s1 = pd.Series(rng.integers(-10, 10, size=len(idx1)), index=idx1)


idx2 = [*"abcd"]
s2 = pd.Series(rng.integers(-10, 10, size=len(idx2)), index=idx2)
print(
    s1,
    s2,
    f"{s1.index.has_duplicates = }",
    f"{s2.index.has_duplicates = }",
    s1 + s2,
    sep="\n",
)


a   -8
b    0
c    1
d   -2
dtype: int64
a     0
b    -1
c     1
d   -10
dtype: int64
s1.index.has_duplicates = False
s2.index.has_duplicates = False
a    -8
b    -1
c     2
d   -12
dtype: int64


In [ ]:
# Not lining up directly will try to line up as much as possible
idx1 = [*"abcd"]
s1 = pd.Series(rng.integers(-10, 10, size=len(idx1)), index=idx1)

# Some overlap
idx2 = [*"bcde"]
s2 = pd.Series(rng.integers(-10, 10, size=len(idx2)), index=idx2)
print(
    s1,
    s2,
    f"{s1.index.has_duplicates = }",
    f"{s2.index.has_duplicates = }",
    s1 + s2,
    sep="\n",
)


a   -8
b   -1
c    5
d    8
dtype: int64
b   -10
c     8
d    -1
e     8
dtype: int64
s1.index.has_duplicates = False
s2.index.has_duplicates = False
a     NaN
b   -11.0
c    13.0
d     7.0
e     NaN
dtype: float64


In [ ]:
# Can check for how this will line up with the Union of get_indexer on both
# -1 won't match anything
print(
    s1,
    s2,
    f"{s1.index.get_indexer(s2.index) = }",
    f"{s2.index.get_indexer(s1.index) = }",
    sep="\n",
)


a   -8
b   -1
c    5
d    8
dtype: int64
b   -10
c     8
d    -1
e     8
dtype: int64
s1.index.get_indexer(s2.index) = array([ 1,  2,  3, -1])
s2.index.get_indexer(s1.index) = array([-1,  0,  1,  2])


In [ ]:
# Can get fancy with how those NaN's are filled
print(
    s1,
    s2,
    f"{pd.Index([*'bcde']).get_indexer(pd.Index([*'abcd']), method='bfill') = }",
    f"{pd.Index([*'bcde']).get_indexer(pd.Index([*'cdef']), method='ffill') = }",
    f"{pd.Index([0, 1, 2, 3]).get_indexer(pd.Index([0, 1.1, 2.2, 3.3]), method='nearest') = }",
    sep="\n",
)


a   -8
b   -1
c    5
d    8
dtype: int64
b   -10
c     8
d    -1
e     8
dtype: int64
pd.Index([*'bcde']).get_indexer(pd.Index([*'abcd']), method='bfill') = array([0, 0, 1, 2])
pd.Index([*'bcde']).get_indexer(pd.Index([*'cdef']), method='ffill') = array([1, 2, 3, 3])
pd.Index([0, 1, 2, 3]).get_indexer(pd.Index([0, 1.1, 2.2, 3.3]), method='nearest') = array([0, 1, 2, 3])


In [ ]:
# What can we pass to this?
signature(s1.add)


<Signature (other, level=None, fill_value=None, axis=0)>

### Presence of Duplicates

If one index had 2 A's, and other had 1 A

Or both had 2 A's

Cartesian product

- Match all of the things on one side to all of the matching things on the other side
- Pairs them up in all combinations

#### Python Counter


In [ ]:
c1 = Counter("abcd")
c2 = Counter("bcde")

print(c1, c2, c1 + c2, sep="\n")


Counter({'a': 1, 'b': 1, 'c': 1, 'd': 1})
Counter({'b': 1, 'c': 1, 'd': 1, 'e': 1})
Counter({'b': 2, 'c': 2, 'd': 2, 'a': 1, 'e': 1})


In [ ]:
# Presence of duplicates on one side
idx1 = [*"abcd"]
s1 = pd.Series(rng.integers(-10, 10, size=len(idx1)), index=idx1)
idx2 = [*"aabc"]
s2 = pd.Series(rng.integers(-10, 10, size=len(idx2)), index=idx2)

print(
    s1,
    s2,
    f"{s1.index.has_duplicates}",
    f"{s2.index.has_duplicates}",
    s1 + s2,
    sep="\n",
)


a    1
b   -9
c   -3
d    3
dtype: int64
a    0
a    2
b   -8
c   -2
dtype: int64
False
True
a     1.0
a     3.0
b   -17.0
c    -5.0
d     NaN
dtype: float64


In [ ]:
# Presence of duplicates on both side
idx1 = [*"aacd"]
s1 = pd.Series(rng.integers(-10, 10, size=len(idx1)), index=idx1)
idx2 = [*"aabc"]
s2 = pd.Series(rng.integers(-10, 10, size=len(idx2)), index=idx2)

print(
    s1,
    s2,
    f"{s1.index.has_duplicates}",
    f"{s2.index.has_duplicates}",
    s1 + s2,
    sep="\n",
)


a     6
a     0
c     5
d   -10
dtype: int64
a    5
a    7
b   -2
c   -1
dtype: int64
True
True
a    11.0
a    13.0
a     5.0
a     7.0
b     NaN
c     4.0
d     NaN
dtype: float64


## No Guarantees

Not a primary key

not guaranteed to be unique, sorted, etc.

Can be disuniform especially with timestamps

Choosing non-unique labels is you choosing to have the metadata that way


In [ ]:
# Series of measurements on a datetime index, disuniform measurements
idx = pd.to_datetime("2000-01-01") + pd.to_timedelta(
    rng.integers(0, 60 * 60, size=1_000).cumsum(), unit="s"
)
s = pd.Series(rng.normal(loc=1, scale=0.1, size=len(idx)).cumprod(), index=idx)
s


2000-01-01 00:03:58    0.929402
2000-01-01 00:24:25    0.902408
2000-01-01 00:46:30    1.035528
2000-01-01 00:47:20    1.193469
2000-01-01 01:42:13    1.288910
                         ...   
2000-01-21 15:48:59    0.066936
2000-01-21 15:50:40    0.065988
2000-01-21 16:22:44    0.073740
2000-01-21 16:51:33    0.078798
2000-01-21 17:29:23    0.073825
Length: 1000, dtype: float64

In [ ]:
# maybe resample for every minute and average them
s.resample("1T").mean()


2000-01-01 00:03:00    0.929402
2000-01-01 00:04:00         NaN
2000-01-01 00:05:00         NaN
2000-01-01 00:06:00         NaN
2000-01-01 00:07:00         NaN
                         ...   
2000-01-21 17:25:00         NaN
2000-01-21 17:26:00         NaN
2000-01-21 17:27:00         NaN
2000-01-21 17:28:00         NaN
2000-01-21 17:29:00    0.073825
Freq: T, Length: 29847, dtype: float64

In [ ]:
# Similar to grouping by and rounding
s.groupby(s.index.round("1T")).mean()


2000-01-01 00:04:00    0.929402
2000-01-01 00:24:00    0.902408
2000-01-01 00:46:00    1.035528
2000-01-01 00:47:00    1.193469
2000-01-01 01:42:00    1.288910
                         ...   
2000-01-21 15:49:00    0.066936
2000-01-21 15:51:00    0.065988
2000-01-21 16:23:00    0.073740
2000-01-21 16:52:00    0.078798
2000-01-21 17:29:00    0.073825
Length: 996, dtype: float64

In [ ]:
# All the things sampled into 1 minute buckets
s.resample("1T").agg(set)


2000-01-01 00:03:00     {0.9294020767894908}
2000-01-01 00:04:00                       {}
2000-01-01 00:05:00                       {}
2000-01-01 00:06:00                       {}
2000-01-01 00:07:00                       {}
                               ...          
2000-01-21 17:25:00                       {}
2000-01-21 17:26:00                       {}
2000-01-21 17:27:00                       {}
2000-01-21 17:28:00                       {}
2000-01-21 17:29:00    {0.07382487922365741}
Freq: T, Length: 29847, dtype: object

In [ ]:
# Index lables of the things sampled into 1 minute buckets (original timestamps)
s.resample("1T").agg(lambda s: {*s.index})


2000-01-01 00:03:00    {2000-01-01 00:03:58}
2000-01-01 00:04:00                       {}
2000-01-01 00:05:00                       {}
2000-01-01 00:06:00                       {}
2000-01-01 00:07:00                       {}
                               ...          
2000-01-21 17:25:00                       {}
2000-01-21 17:26:00                       {}
2000-01-21 17:27:00                       {}
2000-01-21 17:28:00                       {}
2000-01-21 17:29:00    {2000-01-21 17:29:23}
Freq: T, Length: 29847, dtype: object

In [ ]:
# Index lables of the things grouped by rounding down to 1 minute
# Where they don't match up is where we are different
s.groupby(s.index.round("1T")).agg(lambda s: {*s.index})


2000-01-01 00:04:00    {2000-01-01 00:03:58}
2000-01-01 00:24:00    {2000-01-01 00:24:25}
2000-01-01 00:46:00    {2000-01-01 00:46:30}
2000-01-01 00:47:00    {2000-01-01 00:47:20}
2000-01-01 01:42:00    {2000-01-01 01:42:13}
                               ...          
2000-01-21 15:49:00    {2000-01-21 15:48:59}
2000-01-21 15:51:00    {2000-01-21 15:50:40}
2000-01-21 16:23:00    {2000-01-21 16:22:44}
2000-01-21 16:52:00    {2000-01-21 16:51:33}
2000-01-21 17:29:00    {2000-01-21 17:29:23}
Length: 996, dtype: object

In [ ]:
# Groupby will fill with NaN where no data, resample fills with empty set
rs = s.resample("1T").agg(lambda s: {*s.index})
gb = s.groupby(s.index.round("1T")).agg(lambda s: {*s.index})
compare_df = pd.DataFrame({"resample": rs, "groupby": gb})
compare_df


,resample,groupby
2000-01-01 00:03:00,{2000-01-01 00:03:58},NaN
2000-01-01 00:04:00,{},{2000-01-01 00:03:58}
2000-01-01 00:05:00,{},NaN
2000-01-01 00:06:00,{},NaN
2000-01-01 00:07:00,{},NaN
...,...,...
2000-01-21 17:25:00,{},NaN
2000-01-21 17:26:00,{},NaN
2000-01-21 17:27:00,{},NaN
2000-01-21 17:28:00,{},NaN


In [ ]:
# Where groupby and resample match up
comp_idx = compare_df.apply(lambda s: s["resample"] == s["groupby"], axis="columns")
comp_idx


2000-01-01 00:03:00    False
2000-01-01 00:04:00    False
2000-01-01 00:05:00    False
2000-01-01 00:06:00    False
2000-01-01 00:07:00    False
                       ...  
2000-01-21 17:25:00    False
2000-01-21 17:26:00    False
2000-01-21 17:27:00    False
2000-01-21 17:28:00    False
2000-01-21 17:29:00     True
Freq: T, Length: 29847, dtype: bool

In [ ]:
# Get just the matches
comp_idx.where(comp_idx).index


DatetimeIndex(['2000-01-01 00:03:00', '2000-01-01 00:04:00',
               '2000-01-01 00:05:00', '2000-01-01 00:06:00',
               '2000-01-01 00:07:00', '2000-01-01 00:08:00',
               '2000-01-01 00:09:00', '2000-01-01 00:10:00',
               '2000-01-01 00:11:00', '2000-01-01 00:12:00',
               ...
               '2000-01-21 17:20:00', '2000-01-21 17:21:00',
               '2000-01-21 17:22:00', '2000-01-21 17:23:00',
               '2000-01-21 17:24:00', '2000-01-21 17:25:00',
               '2000-01-21 17:26:00', '2000-01-21 17:27:00',
               '2000-01-21 17:28:00', '2000-01-21 17:29:00'],
              dtype='datetime64[ns]', length=29847, freq='T')

In [ ]:
# Original samples' index labels
same_idx = comp_idx.loc[comp_idx].index
same_idx


DatetimeIndex(['2000-01-01 00:24:00', '2000-01-01 00:46:00',
               '2000-01-01 00:47:00', '2000-01-01 01:42:00',
               '2000-01-01 03:01:00', '2000-01-01 03:11:00',
               '2000-01-01 05:05:00', '2000-01-01 05:34:00',
               '2000-01-01 05:39:00', '2000-01-01 06:57:00',
               ...
               '2000-01-21 03:48:00', '2000-01-21 04:32:00',
               '2000-01-21 06:59:00', '2000-01-21 08:55:00',
               '2000-01-21 09:56:00', '2000-01-21 11:42:00',
               '2000-01-21 13:35:00', '2000-01-21 14:20:00',
               '2000-01-21 15:38:00', '2000-01-21 17:29:00'],
              dtype='datetime64[ns]', length=501, freq=None)

In [ ]:
# yup, they all match when we look them up
(rs.loc[same_idx] == gb.loc[same_idx]).all()


True

In [ ]:
# Groupby and Resample labels
common_idx = rs.index.intersection(gb.index)
common_idx


DatetimeIndex(['2000-01-01 00:04:00', '2000-01-01 00:24:00',
               '2000-01-01 00:46:00', '2000-01-01 00:47:00',
               '2000-01-01 01:42:00', '2000-01-01 02:14:00',
               '2000-01-01 02:36:00', '2000-01-01 03:01:00',
               '2000-01-01 03:11:00', '2000-01-01 04:00:00',
               ...
               '2000-01-21 13:35:00', '2000-01-21 14:20:00',
               '2000-01-21 14:33:00', '2000-01-21 15:01:00',
               '2000-01-21 15:38:00', '2000-01-21 15:49:00',
               '2000-01-21 15:51:00', '2000-01-21 16:23:00',
               '2000-01-21 16:52:00', '2000-01-21 17:29:00'],
              dtype='datetime64[ns]', length=996, freq=None)

In [ ]:
# Original indices they both have, without the ones that are matches
diff_idx = common_idx.difference(same_idx)
diff_idx


DatetimeIndex(['2000-01-01 00:04:00', '2000-01-01 02:14:00',
               '2000-01-01 02:36:00', '2000-01-01 04:00:00',
               '2000-01-01 04:30:00', '2000-01-01 05:01:00',
               '2000-01-01 06:36:00', '2000-01-01 06:52:00',
               '2000-01-01 09:00:00', '2000-01-01 09:43:00',
               ...
               '2000-01-21 10:36:00', '2000-01-21 11:23:00',
               '2000-01-21 12:02:00', '2000-01-21 12:54:00',
               '2000-01-21 14:33:00', '2000-01-21 15:01:00',
               '2000-01-21 15:49:00', '2000-01-21 15:51:00',
               '2000-01-21 16:23:00', '2000-01-21 16:52:00'],
              dtype='datetime64[ns]', length=495, freq=None)

In [ ]:
# yeah, everything at those locations are different
# So groupby and resample took different steps to calculate the same things
(rs.loc[diff_idx] != gb.loc[diff_idx]).all()


True

## Beyond the Series

Underneath series is data


In [ ]:
# df with some fake stock ticker data
tickers = rng.choice([*ascii_lowercase], size=(100, 4)).view("<U4").ravel()
dates = pd.date_range("2000-01-01", "2000-12-31", name="date")
prices = (
    100
    * rng.random(size=len(tickers))
    * rng.normal(loc=1, scale=0.01, size=(len(dates), len(tickers))).cumprod(axis=0)
).ravel()
volumes = rng.integers(-50_000, 50_000, size=len(dates) * len(tickers)).round(-2)

df = pd.DataFrame(
    {
        "date": np.repeat(dates, len(tickers)),
        "ticker": np.tile(tickers, len(dates)),
        "price": prices,
        "volume": volumes,
    }
)

# First 3 entries of data
df.head(3)


,date,ticker,price,volume
0,2000-01-01,ahop,96.274372,46200
1,2000-01-01,okvd,45.211626,-37000
2,2000-01-01,vsrw,70.157305,-44300


In [ ]:
# data at index label 0, column 'ticker'
df.loc[0, "ticker"]


'ahop'

In [ ]:
# data at label 0 and column 0
df.iloc[0, 0]


Timestamp('2000-01-01 00:00:00')

In [ ]:
# data at label 0, then go into ticker of that?
# Lookup in dataframe reduced to series, then another lookup
df.loc[0]["ticker"]
# df.loc[0].loc['ticker'] # Same thing, less ambiguous


'ahop'

In [ ]:
df.loc[0]


date      2000-01-01 00:00:00
ticker                   ahop
price               96.274372
volume                  46200
Name: 0, dtype: object

In [ ]:
# Give me every row, just the ticker column
df.loc[:, "ticker"]


0        ahop
1        okvd
2        vsrw
3        lpjy
4        agnr
         ... 
36595    fnbo
36596    nkzj
36597    gwir
36598    kqca
36599    kajr
Name: ticker, Length: 36600, dtype: object

### DataFrame is doubly indexed

Series is singly-indexed one-dimensional data

DataFrame is doubly-indexed

Store a bunch of Series' together and line them up

DataFrame has 1 index for getting the row,
and 1 index for getting the Series

Technically not 2-dimensional:

- Cannot select diagonals
- Data Fundamentally changes on rotation

df of stock data is 4 datasets, all aligned on the date:

- 'date'
- 'ticker'
- 'price'
- 'volume'

But a ticker and a date are labels....

Now it is like 2 datasets, collected by date and ticker


In [ ]:
df = pd.DataFrame(
    {
        "date": np.repeat(dates, len(tickers)),
        "ticker": np.tile(tickers, len(dates)),
        "price": prices,
        "volume": volumes,
    }
).set_index(["date", "ticker"])
df.head(3)


price  volume
date       ticker                   
2000-01-01 ahop    96.274372   46200
           okvd    45.211626  -37000
           vsrw    70.157305  -44300

## Indexed One-Dimensional Data

One set of One-dimensional data: Series
multiple sets of One-dimensional data: DataFrame

When the subset of the DataFrame is one-dimensional, return Series.
Else return DataFrame


In [ ]:
df = pd.DataFrame(rng.normal(size=(5, 3)).round(2))
df


,0,1,2
0,0.57,-0.26,0.63
1,-1.21,0.22,-0.62
2,1.07,-0.07,-0.92
3,-1.17,0.16,1.07
4,-0.35,0.87,1.42


In [ ]:
# Indexing out a row returns a Series
print(df.loc[0], type(df.loc[0]), sep="\n")


0    0.57
1   -0.26
2    0.63
Name: 0, dtype: float64
<class 'pandas.core.series.Series'>


In [ ]:
# Column is Series as well
print(df[0], type(df[0]), sep="\n")


0    0.57
1   -1.21
2    1.07
3   -1.17
4   -0.35
Name: 0, dtype: float64
<class 'pandas.core.series.Series'>


### Loc can return dataframe


In [ ]:
df = pd.DataFrame(rng.normal(size=(5, 3)).round(2))
print(df.loc[0], df.loc[2], sep="\n")


0    1.05
1   -0.59
2   -0.77
Name: 0, dtype: float64
0    0.09
1    0.81
2   -1.87
Name: 2, dtype: float64


In [ ]:
# Putting duplicates in index
idx = [0, 0, 1, 1, 2]
df = pd.DataFrame(rng.normal(size=(len(idx), 3)).round(2), index=idx)
df.columns = [0, 1, 1]

print(df.loc[0], type(df.loc[0]), df.loc[2], type(df.loc[2]), sep="\n")


      0     1     1
0 -0.73 -0.25 -0.07
0  0.35  0.29 -0.93
<class 'pandas.core.frame.DataFrame'>
0    0.39
1   -1.30
1    0.54
Name: 2, dtype: float64
<class 'pandas.core.series.Series'>


In [ ]:
# Putting duplicates in index
idx = [0, 0, 1, 1, 2]
df = pd.DataFrame(rng.normal(size=(len(idx), 3)).round(2), index=idx)
df.columns = [0, 1, 1]

print(df.loc[0], type(df.loc[0]), df.loc[2], type(df.loc[2]), sep="\n")


      0     1     1
0 -0.04  0.77  1.61
0 -0.25 -0.77  2.16
<class 'pandas.core.frame.DataFrame'>
0   -0.52
1    0.81
1    0.64
Name: 2, dtype: float64
<class 'pandas.core.series.Series'>


### iloc is unambiguous about numbers of selection


In [ ]:
# iloc is unambiguous
print(df.iloc[0], type(df.iloc[0]), sep="\n")


0   -0.04
1    0.77
1    1.61
Name: 0, dtype: float64
<class 'pandas.core.series.Series'>


### Selecting from one index yields a chunk of the perpendicular index


In [ ]:
# Want the column index when we look into the rows
print(df.iloc[0], df.iloc[0].index, df.iloc[0].name, sep="\n")


0   -0.04
1    0.77
1    1.61
Name: 0, dtype: float64
Int64Index([0, 1, 1], dtype='int64')
0


In [ ]:
# Name is by label
print(df.iloc[2], df.iloc[2].index, df.iloc[2].name, sep="\n")


0   -0.64
1    0.51
1   -1.32
Name: 1, dtype: float64
Int64Index([0, 1, 1], dtype='int64')
1


In [ ]:
# Bracket implicit behaviour tries its best to guess
print(df[0], df[0].index, df[0].name, sep="\n")


0   -0.04
0   -0.25
1   -0.64
1   -1.56
2   -0.52
Name: 0, dtype: float64
Int64Index([0, 0, 1, 1, 2], dtype='int64')
0


In [ ]:
df.iloc[0][0]  # By label


-0.04

In [ ]:
df[0][3:]  # By position


1   -1.56
2   -0.52
Name: 0, dtype: float64

## Setting with Copy Makes sense


In [ ]:
# df with different types of data
df = pd.DataFrame(
    {
        "a": rng.integers(-10, 10, size=(size := 5)),
        "b": rng.normal(size=size).round(2),
        "c": rng.choice([*ascii_lowercase], size=size),
    }
)
df


,a,b,c
0,8,-0.08,s
1,-3,2.02,b
2,-3,0.72,k
3,4,-0.22,c
4,-1,0.52,a


In [ ]:
# Multiple datasets, not guaranteed to have same types
# This result has to be object type if they don't all match
df.iloc[0]


a       8
b   -0.08
c       s
Name: 0, dtype: object

In [ ]:
type(df.iloc[0])


pandas.core.series.Series

In [ ]:
df.iloc[0].dtype


dtype('O')

### Iloc making a copy


In [ ]:
# This has to make a copy
with catch_warnings():
    filterwarnings("error")
    df.iloc[0]["c"] = "z"


SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [ ]:
# Oh, we just make it one bracket, right...?
# 'c' isn't an integer though...
with catch_warnings():
    filterwarnings("error")
    df.iloc[0, "c"] = "z"


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
# Oh, need loc? but that behaves different sometimes
df = pd.DataFrame(
    {
        "a": rng.integers(-10, 10, size=(size := 5)),
        "b": rng.normal(size=size).round(2),
        "c": rng.choice([*ascii_lowercase], size=size),
    }
)

with catch_warnings():
    filterwarnings("error")
    df.loc[0, "c"] = "z"
df


,a,b,c
0,-10,0.86,z
1,9,-0.24,l
2,9,-2.03,g
3,-4,0.12,v
4,1,-0.51,d


In [ ]:
# To be unambiguous need to ask the index of the columns for a translation
df = pd.DataFrame(
    {
        "a": rng.integers(-10, 10, size=(size := 5)),
        "b": rng.normal(size=size).round(2),
        "c": rng.choice([*ascii_lowercase], size=size),
    }
)

with catch_warnings():
    filterwarnings("error")
    df.iloc[0, df.columns.get_loc("c")] = "z"
df


,a,b,c
0,6,0.57,z
1,-8,0.68,z
2,-6,0.42,v
3,3,0.20,w
4,-6,0.55,y


In [ ]:
# or ask the index of the rows for a translation
df = pd.DataFrame(
    {
        "a": rng.integers(-10, 10, size=(size := 5)),
        "b": rng.normal(size=size).round(2),
        "c": rng.choice([*ascii_lowercase], size=size),
    }
)

with catch_warnings():
    filterwarnings("error")
    df.loc[df.index.get_loc(0), "c"] = "z"
df


,a,b,c
0,6,0.57,z
1,-8,0.68,z
2,-6,0.42,v
3,3,0.20,w
4,-6,0.55,y


## DataFrame is typified by a Block Manager


In [ ]:
df = pd.DataFrame(
    {
        "a": rng.integers(-10, 10, size=(size := 5)),
        "b": rng.normal(size=size).round(2),
        "c": rng.choice([*ascii_lowercase], size=size),
    }
)
df


,a,b,c
0,-9,-0.40,w
1,4,-0.04,r
2,-9,-0.35,f
3,-5,0.43,p
4,6,-1.21,y


In [ ]:
# Underlying data
df._data


BlockManager
Items: Index(['a', 'b', 'c'], dtype='object')
Axis 1: RangeIndex(start=0, stop=5, step=1)
NumericBlock: slice(0, 1, 1), 1 x 5, dtype: int64
NumericBlock: slice(1, 2, 1), 1 x 5, dtype: float64
ObjectBlock: slice(2, 3, 1), 1 x 5, dtype: object

In [ ]:
# Way to refer to row labels (major axis)
df.index


RangeIndex(start=0, stop=5, step=1)

In [ ]:
# Way to refer to column labels (minor axis)
df.columns


Index(['a', 'b', 'c'], dtype='object')

In [ ]:
# Double loc example
df = pd.DataFrame(
    data=rng.normal(size=(num_rows := 5, num_cols := 5)).round(2),
    columns=pd.period_range("2000Q1", periods=num_cols, freq="Q"),
    index=[*ascii_lowercase[:num_rows]],
)

print(df, df.index, df.columns, df.loc["a"].loc["2000-03-31":"2000-07-12"], sep="\n\n")


   2000Q1  2000Q2  2000Q3  2000Q4  2001Q1
a    0.66   -0.56    0.10   -0.81   -0.29
b    2.17   -1.21    1.05    0.02    2.16
c   -1.37   -0.89    2.26    0.37   -1.09
d   -1.43    0.09   -1.70   -0.37   -1.55
e    1.10    1.84   -0.12    0.82   -1.08

Index(['a', 'b', 'c', 'd', 'e'], dtype='object')

PeriodIndex(['2000Q1', '2000Q2', '2000Q3', '2000Q4', '2001Q1'], dtype='period[Q-DEC]')

2000Q1    0.66
2000Q2   -0.56
2000Q3    0.10
Freq: Q-DEC, Name: a, dtype: float64


## DataFrame alignment


In [ ]:
# Like indexed similar data
# Matches ROWS and COLUMNS
# Will make nan
df1 = pd.DataFrame(
    {
        "a": rng.integers(-10, 10, size=(size := 5)),
        "b": rng.integers(-10, 10, size=5),
        "c": rng.integers(-10, 10, size=5),
    },
    index=pd.date_range("2000-01-01", periods=size, name="date"),
)
df2 = pd.DataFrame(
    {
        "a": rng.integers(-10, 10, size=(size := 5)),
        "b": rng.integers(-10, 10, size=5),
        "c": rng.integers(-10, 10, size=5),
    },
    index=pd.date_range("2000-01-02", periods=size, name="date"),
)

print(df1, df2, df1 + df2, sep="\n\n")


            a   b  c
date                
2000-01-01 -9 -10 -1
2000-01-02 -7  -9  0
2000-01-03 -4   3 -4
2000-01-04 -6  -8 -2
2000-01-05  7  -1 -9

            a  b  c
date               
2000-01-02 -8  7  5
2000-01-03  7 -3 -2
2000-01-04 -5  4  5
2000-01-05 -2 -4  0
2000-01-06 -6 -1 -4

               a    b    c
date                      
2000-01-01   NaN  NaN  NaN
2000-01-02 -15.0 -2.0  5.0
2000-01-03   3.0  0.0 -6.0
2000-01-04 -11.0 -4.0  3.0
2000-01-05   5.0 -5.0 -9.0
2000-01-06   NaN  NaN  NaN


In [ ]:
# Extra column causes all of the row, col matchups to mismatch in the new column
df1 = pd.DataFrame(
    {
        "a": rng.integers(-10, 10, size=(size := 5)),
        "b": rng.integers(-10, 10, size=5),
        "c": rng.integers(-10, 10, size=5),
    },
    index=pd.date_range("2000-01-01", periods=size, name="date"),
)
df2 = pd.DataFrame(
    {
        "a": rng.integers(-10, 10, size=5),
        "b": rng.integers(-10, 10, size=5),
        "c": rng.integers(-10, 10, size=5),
        "d": rng.integers(-10, 10, size=5),
    },
    index=pd.date_range("2000-01-02", periods=size, name="date"),
)

print(df1, df2, df1 + df2, sep="\n\n")


            a  b  c
date               
2000-01-01  9 -8  4
2000-01-02 -8 -5  6
2000-01-03 -8  8 -5
2000-01-04  7 -1  0
2000-01-05  1 -6 -8

             a  b  c  d
date                   
2000-01-02  -4  4  8  0
2000-01-03 -10 -8  4 -6
2000-01-04   8 -8  9 -5
2000-01-05  -1 -9 -9 -6
2000-01-06  -1  3  7  7

               a     b     c   d
date                            
2000-01-01   NaN   NaN   NaN NaN
2000-01-02 -12.0  -1.0  14.0 NaN
2000-01-03 -18.0   0.0  -1.0 NaN
2000-01-04  15.0  -9.0   9.0 NaN
2000-01-05   0.0 -15.0 -17.0 NaN
2000-01-06   NaN   NaN   NaN NaN


### Series Matching


In [ ]:
# Match columns of dataframe to Rows of Series
# This Series has integers, columns are "a, "b", "c" -> NaN
df = pd.DataFrame(
    {
        "a": rng.integers(-10, 10, size=(size := 5)),
        "b": rng.integers(-10, 10, size=5),
        "c": rng.integers(-10, 10, size=5),
    },
    index=pd.date_range("2000-01-01", periods=size, name="date"),
)
s = pd.Series(rng.integers(-10, 10, size=len(df)), index=df.index)

print(df, s, sep="\n\n")


            a  b   c
date                
2000-01-01  6 -8   5
2000-01-02  2 -9   1
2000-01-03  0  9   7
2000-01-04  5  7 -10
2000-01-05 -7  4   0

date
2000-01-01   -3
2000-01-02    7
2000-01-03   -8
2000-01-04   -8
2000-01-05    3
Freq: D, dtype: int64


In [ ]:
df + s


,2000-01-01 00:00:00,2000-01-02 00:00:00,2000-01-03 00:00:00,2000-01-04 00:00:00,2000-01-05 00:00:00,a,b,c
date,,,,,,,,
2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# order doesn't matter
s + df


,2000-01-01 00:00:00,2000-01-02 00:00:00,2000-01-03 00:00:00,2000-01-04 00:00:00,2000-01-05 00:00:00,a,b,c
date,,,,,,,,
2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Can't transpose a series, that just returns the series
df + s.T


,2000-01-01 00:00:00,2000-01-02 00:00:00,2000-01-03 00:00:00,2000-01-04 00:00:00,2000-01-05 00:00:00,a,b,c
date,,,,,,,,
2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Getting clever won't necessarily work
(df.T + s).T


,a,b,c
date,,,
2000-01-01,3,-11,2
2000-01-02,9,-2,8
2000-01-03,-8,1,-1
2000-01-04,-3,-1,-18
2000-01-05,-4,7,3


In [ ]:
# Make series use df columns
df = pd.DataFrame(
    {
        "a": rng.integers(-10, 10, size=(size := 5)),
        "b": rng.integers(-10, 10, size=5),
        "c": rng.integers(-10, 10, size=5),
    },
    index=pd.date_range("2000-01-01", periods=size, name="date"),
)
s = pd.Series(rng.integers(-10, 10, size=len(df.columns)), index=df.columns)

print(df, s, sep="\n\n")


            a   b   c
date                 
2000-01-01 -5   7   0
2000-01-02 -1   1  -5
2000-01-03  4 -10  -4
2000-01-04 -2   4   1
2000-01-05  2   1 -10

a   -6
b   -4
c    0
dtype: int64


In [ ]:
# Does the add
df + s


,a,b,c
date,,,
2000-01-01,-11,3,0
2000-01-02,-7,-3,-5
2000-01-03,-2,-14,-4
2000-01-04,-8,0,1
2000-01-05,-4,-3,-10


### Back to Real world data


In [ ]:
# df with some fake stock ticker data
tickers = rng.choice([*ascii_lowercase], size=(100, 4)).view("<U4").ravel()
dates = pd.date_range("2000-01-01", "2000-12-31", name="date")
prices = (
    100
    * rng.random(size=len(tickers))
    * rng.normal(loc=1, scale=0.01, size=(len(dates), len(tickers))).cumprod(axis=0)
).ravel()
volumes = rng.integers(-50_000, 50_000, size=len(dates) * len(tickers)).round(-2)

df = (
    pd.DataFrame(
        {
            "date": np.repeat(dates, len(tickers)),
            "ticker": np.tile(tickers, len(dates)),
            "price": prices,
            "volume": volumes,
        }
    )
    .set_index(["date", "ticker"])
    .sort_index()
)

factor = pd.Series(
    rng.normal(loc=1, scale=0.1, size=len(tickers)), index=tickers, name="factor"
)
factor.index.name = "ticker"

df


price  volume
date       ticker                   
2000-01-01 aapa    28.629523   -6500
           acio    26.599490   31600
           azxb    27.281616    8600
           bbcy    56.185786    6000
           bedw    47.453226     800
...                      ...     ...
2000-12-31 zdsu    13.818191   39700
           zrea    11.467948   -1000
           zrqp     4.150418   29800
           zwbx    55.972953  -11000
           zzwc    16.267819  -14500

[36600 rows x 2 columns]

In [ ]:
# factor will be multiplied to each ticker, but we only want to apply to price
factor


ticker
zwbx    1.069143
vxie    1.030164
mubh    0.995270
kvle    0.844614
srpt    1.159280
          ...   
ogrf    0.903039
yhrf    0.976699
xnei    0.752632
mrkc    0.954370
bbcy    0.977107
Name: factor, Length: 100, dtype: float64

In [ ]:
# So pull the column? and work down each value?
df["price"] * factor


date        ticker
2000-01-01  aapa      27.557897
            acio      23.711216
            azxb      29.128398
            bbcy      54.899523
            bedw      40.594128
                        ...    
2000-12-31  zdsu      12.344960
            zrea      11.933056
            zrqp       4.467614
            zwbx      59.843115
            zzwc      16.878471
Length: 36600, dtype: float64

In [ ]:
# multi indexed data frame, pull out just 'aapa' ticker, but all dates, all columns
df.loc[pd.IndexSlice[:, "aapa", :]].sample(3)


,price,volume
date,,
2000-02-07,27.882312,42900
2000-09-29,24.164270,-14700
2000-10-22,22.856649,-49900


In [ ]:
# df['price'] is multi indexed too!
df["price"]


date        ticker
2000-01-01  aapa      28.629523
            acio      26.599490
            azxb      27.281616
            bbcy      56.185786
            bedw      47.453226
                        ...    
2000-12-31  zdsu      13.818191
            zrea      11.467948
            zrqp       4.150418
            zwbx      55.972953
            zzwc      16.267819
Name: price, Length: 36600, dtype: float64

## What is MultiIndex and how do they align?


In [ ]:
# MultiIndex series with symbols and quarters
s = pd.Series(
    index=(
        idx := pd.MultiIndex.from_product(
            [
                pd.period_range("2000Q1", periods=4, freq="Q", name="quarter"),
                ["abc", "def", "xyz"],
            ],
            names=["quarter", "symbol"],
        )
    ),
    data=rng.integers(-10, 10, size=len(idx)),
)

s


quarter  symbol
2000Q1   abc       1
         def       3
         xyz      -8
2000Q2   abc      -3
         def       2
         xyz      -5
2000Q3   abc      -3
         def      -4
         xyz      -3
2000Q4   abc      -2
         def       6
         xyz      -8
dtype: int64

In [ ]:
# Can look at everything from Q1
s.loc["2000Q1"]


symbol
abc    1
def    3
xyz   -8
dtype: int64

In [ ]:
# Or a specific ticker
s.loc["2000Q1", "abc"]


1

In [ ]:
# Or select a few columns in particular from if it was a dataframe
# NOTE tuple seems to try to hash as an index label ('2000Q1', 'abc')
s.loc[["2000Q1", "abc"], :]


/tmp/ipykernel_373/339101015.py:3: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  s.loc[['2000Q1', 'abc'], :]


quarter  symbol
2000Q1   abc       1
         def       3
         xyz      -8
dtype: int64

In [ ]:
# Avoids repeating a lot of looping code. Here figures out which quarter has March 15th, then slices all quarters ging forward.
# Only picks 'abc' and 'xyz' symbols
s.loc[pd.IndexSlice["2000-03-15":, ["abc", "xyz"]]]


quarter  symbol
2000Q1   abc       1
         xyz      -8
2000Q2   abc      -3
         xyz      -5
2000Q3   abc      -3
         xyz      -3
2000Q4   abc      -2
         xyz      -8
dtype: int64

### MultiIndex not only hierarchical index

Syntax is just a little different.

Datetime is basically multiindex of year, month, day, hour, min, second (down to your level of time fidelity)


In [ ]:
# datetime index is hierarchical too!
s = pd.Series(
    index=(idx := pd.date_range("2000-01-01", periods=6, name="date", freq="14D")),
    data=rng.integers(-10, 10, size=len(idx)),
)
s


date
2000-01-01    8
2000-01-15    0
2000-01-29    3
2000-02-12   -7
2000-02-26   -2
2000-03-11    6
Freq: 14D, dtype: int64

In [ ]:
# Selection is based on year
s.loc["2000"]


date
2000-01-01    8
2000-01-15    0
2000-01-29    3
2000-02-12   -7
2000-02-26   -2
2000-03-11    6
Freq: 14D, dtype: int64

In [ ]:
# Or go one layer lower to months
s.loc["2000-01"]


date
2000-01-01    8
2000-01-15    0
2000-01-29    3
Freq: 14D, dtype: int64

## Ops in Pandas == Index Ops

Everything that happens to a Series or DataFrame can be discussed in terms of how the Index(es) are manipulated

### Agg vs Transform vs Apply

Apply works on df, Agg and Transform work on Series.

More to cover on that topic


In [ ]:
s = pd.Series(
    index=(idx := pd.date_range("2000-01-01", periods=1_000, name="date", freq="D")),
    data=rng.integers(-10, 10, size=len(idx)),
)
s


date
2000-01-01    6
2000-01-02   -1
2000-01-03    2
2000-01-04    8
2000-01-05   -1
             ..
2002-09-22    3
2002-09-23   -8
2002-09-24    2
2002-09-25    8
2002-09-26    2
Freq: D, Length: 1000, dtype: int64

In [ ]:
# Can group by month number, not really what we want
s.groupby(s.index.month).mean()


date
1     0.139785
2    -1.023529
3    -0.086022
4     0.144444
5    -1.881720
6    -0.788889
7     0.752688
8    -0.612903
9    -1.465116
10   -0.193548
11   -0.650000
12    0.209677
dtype: float64

In [ ]:
# or change to monthly
s.groupby(s.index.to_period("M")).mean()


date
2000-01    1.387097
2000-02   -2.413793
2000-03    0.612903
2000-04   -0.400000
2000-05   -3.225806
2000-06   -1.200000
2000-07    1.129032
2000-08   -0.354839
2000-09   -3.000000
2000-10    0.387097
2000-11   -0.100000
2000-12   -0.935484
2001-01   -0.354839
2001-02   -1.500000
2001-03   -0.774194
2001-04    1.500000
2001-05   -0.612903
2001-06   -1.100000
2001-07   -0.322581
2001-08    0.322581
2001-09   -0.666667
2001-10   -0.774194
2001-11   -1.200000
2001-12    1.354839
2002-01   -0.612903
2002-02    0.892857
2002-03   -0.096774
2002-04   -0.666667
2002-05   -1.806452
2002-06   -0.066667
2002-07    1.451613
2002-08   -1.806452
2002-09   -0.615385
Freq: M, dtype: float64

In [ ]:
# Which can be accessed in normal date lookup
s.groupby(s.index.to_period("M")).mean().loc["2000-02-14":"2000-05-01"]


date
2000-02   -2.413793
2000-03    0.612903
2000-04   -0.400000
2000-05   -3.225806
Freq: M, dtype: float64

In [ ]:
# Agg takes the groups into one scalar value. Something indexed on each of the groups. One value for each
s.groupby(s.index.to_period("M")).agg(lambda s: s.skew())
s.groupby(s.index.to_period("M")).agg(kurtosis)


date
2000-01   -1.123975
2000-02   -0.996559
2000-03   -0.965931
2000-04   -1.066107
2000-05   -0.989043
2000-06   -1.168856
2000-07   -1.098912
2000-08   -1.149583
2000-09   -0.965930
2000-10   -0.943441
2000-11   -1.439003
2000-12   -1.081764
2001-01   -1.218814
2001-02   -1.214105
2001-03   -1.300023
2001-04   -0.598473
2001-05   -1.465577
2001-06   -0.980962
2001-07   -1.031225
2001-08   -1.259753
2001-09   -1.178845
2001-10   -0.873639
2001-11   -1.119199
2001-12   -0.983462
2002-01   -1.424304
2002-02   -1.287963
2002-03   -1.179984
2002-04   -1.042918
2002-05   -1.190532
2002-06   -1.200387
2002-07   -1.190984
2002-08   -1.145385
2002-09   -1.137459
Freq: M, dtype: float64

In [ ]:
# Transform keeps the original data shape and index
# Kind of like apply but more about 1-1 mutation
s.groupby(s.index.to_period("M")).transform(zscore)


date
2000-01-01    0.780315
2000-01-02   -0.403799
2000-01-03    0.103678
2000-01-04    1.118633
2000-01-05   -0.403799
                ...   
2002-09-22    0.630036
2002-09-23   -1.286882
2002-09-24    0.455771
2002-09-25    1.501362
2002-09-26    0.455771
Freq: D, Length: 1000, dtype: float64

In [ ]:
# Apply will make us a whole new index
# Then concatenate results into a brand new DataFrame
s.groupby(s.index.to_period("M")).apply(lambda df: df[df > 0].cumsum())


date     date      
2000-01  2000-01-01     6
         2000-01-03     8
         2000-01-04    16
         2000-01-07    20
         2000-01-09    29
                       ..
2002-09  2002-09-21    41
         2002-09-22    44
         2002-09-24    46
         2002-09-25    54
         2002-09-26    56
Length: 453, dtype: int64

### DataFrame API

Ops in terms of index


In [ ]:
tickers = rng.choice([*ascii_lowercase], size=(100, 4)).view("<U4").ravel()
dates = pd.date_range("2000-01-01", "2000-12-31", name="date")
prices = (
    100
    * rng.random(size=len(tickers))
    * rng.normal(loc=1, scale=0.01, size=(len(dates), len(tickers))).cumprod(axis=0)
).ravel()
volumes = rng.integers(-50_000, 50_000, size=len(dates) * len(tickers)).round(-2)

df = (
    pd.DataFrame(
        {
            "date": np.repeat(dates, len(tickers)),
            "ticker": np.tile(tickers, len(dates)),
            "price": prices,
            "volume": volumes,
        }
    )
    .set_index(["date", "ticker"])
    .sort_index()
)

df


price  volume
date       ticker                    
2000-01-01 aljf      5.890248    3000
           azze     47.526226   33800
           bcfx     27.281503  -18400
           bura      9.399954   29900
           bxjr     43.430960   46600
...                       ...     ...
2000-12-31 ynua    106.024816    3700
           zofz     40.006408   30500
           zpbs     66.899076   -2400
           zpou     94.937277  -40300
           zvur     19.810396   22800

[36600 rows x 2 columns]

In [ ]:
# Fetch from first 3 index values
df.head(3)


In [ ]:
# Not very indexy attributes
df.ndim, df.shape


(2, (36600, 2))

In [ ]:
df.flags, df.attrs


(<Flags(allows_duplicate_labels=True)>, {})

In [ ]:
df.index, df.columns, df.axes


(MultiIndex([('2000-01-01', 'aljf'),
             ('2000-01-01', 'azze'),
             ('2000-01-01', 'bcfx'),
             ('2000-01-01', 'bura'),
             ('2000-01-01', 'bxjr'),
             ('2000-01-01', 'byjm'),
             ('2000-01-01', 'chwl'),
             ('2000-01-01', 'cood'),
             ('2000-01-01', 'ddej'),
             ('2000-01-01', 'devn'),
             ...
             ('2000-12-31', 'xqlv'),
             ('2000-12-31', 'xxgz'),
             ('2000-12-31', 'xzws'),
             ('2000-12-31', 'yeyy'),
             ('2000-12-31', 'ykzx'),
             ('2000-12-31', 'ynua'),
             ('2000-12-31', 'zofz'),
             ('2000-12-31', 'zpbs'),
             ('2000-12-31', 'zpou'),
             ('2000-12-31', 'zvur')],
            names=['date', 'ticker'], length=36600),
 Index(['price', 'volume'], dtype='object'),
 [MultiIndex([('2000-01-01', 'aljf'),
              ('2000-01-01', 'azze'),
              ('2000-01-01', 'bcfx'),
              ('2000-01-01', '

In [ ]:
# Take layer of a multi index and pivot into columns as the new innermost layer
# Provides column for each ticker for each time
df.unstack()


price                                                        \
ticker          aljf       azze       bcfx      bura       bxjr       byjm   
date                                                                         
2000-01-01  5.890248  47.526226  27.281503  9.399954  43.430960  67.812785   
2000-01-02  5.852996  47.693972  27.306171  9.511146  43.200433  68.550336   
2000-01-03  5.844182  47.448944  27.154015  9.385744  42.987742  68.384636   
2000-01-04  5.808688  47.645107  27.390296  9.481842  43.260356  67.975265   
2000-01-05  5.831615  47.816893  27.413597  9.420158  43.188337  68.481892   
...              ...        ...        ...       ...        ...        ...   
2000-12-27  6.607238  55.192695  24.831951  9.439867  56.164478  65.475797   
2000-12-28  6.543884  55.039633  24.278309  9.325261  55.701233  67.114242   
2000-12-29  6.517716  55.624461  24.598243  9.128540  55.706000  68.749210   
2000-12-30  6.591016  55.667094  24.788863  9.263568  54.772754  67.988267   
2000-12-31  6.574865  55.602771  24.752609  9.401429  54.131100  67.795625   

                                                         ... volume         \
ticker           chwl        cood       ddej       devn  ...   xqlv   xxgz   
date                                                     ...                 
2000-01-01  41.721608  100.842358  93.490675  65.595932  ...  46100  40700   
2000-01-02  41.943444  101.992000  92.149006  65.403305  ...  22700  40100   
2000-01-03  42.593299  103.356012  92.389723  65.549089  ...  32200    700   
2000-01-04  42.937935  102.147746  92.899474  65.685102  ...  -3100  23700   
2000-01-05  43.231657  101.986142  92.110595  65.969694  ...  43500 -39800   
...               ...         ...        ...        ...  ...    ...    ...   
2000-12-27  41.647091  103.301653  86.940768  52.992312  ...  13100  -6400   
2000-12-28  41.589839  103.350904  86.762329  53.230137  ...  43600  -6700   
2000-12-29  41.440779  103.767453  85.836942  53.577230  ...  14000  46000   
2000-12-30  41.662555  104.415893  85.742283  53.057292  ... -15200 -24700   
2000-12-31  41.481638  105.581286  86.170840  52.583008  ...  -4600 -14700   

                                                                    
ticker       xzws   yeyy   ykzx   ynua   zofz   zpbs   zpou   zvur  
date                                                                
2000-01-01 -45600   8800 -48900  -7400  -3600  48000  -7400  43700  
2000-01-02  33000  12100  12600 -17500   -300 -24100 -22700  -6200  
2000-01-03  38900 -33400 -37200 -33700 -11700  -9100 -37000  23000  
2000-01-04  34600  24400  29800 -45700 -37100  -8800 -21000 -43800  
2000-01-05  32200   -900 -18500 -39400    400 -28900  32900  25600  
...           ...    ...    ...    ...    ...    ...    ...    ...  
2000-12-27 -21700  -3300  24800 -46700 -22200  -5900 -50000  43000  
2000-12-28 -36100  50000  36200   2100 -22900  20900  43600  23900  
2000-12-29 -22700   5100 -35100 -19800  23000 -19000 -12900  32000  
2000-12-30 -48100  36200 -49800   5200 -49800  -1000 -32100  41100  
2000-12-31 -30300 -38300  20300   3700  30500  -2400 -40300  22800  

[366 rows x 200 columns]

In [ ]:
# Stack takes one layer of the column multi index and puts it as innermost layer of row in
# Awkward in this case. Sometimes nice for comparison
# Dual of unstack
df.stack()


0  a   -10
   b    -9
   c   -10
1  a    -7
   b     6
   c     2
2  a     8
   b     0
   c     2
3  a     9
   b     4
   c     2
4  a     0
   b     1
   c     8
dtype: int64

In [ ]:
# Imagine some excel data
# Each row is a ticker, they entered the data
df = pd.DataFrame(
    {
        "sector": rng.choice("tech healthcare energy".split(), size=(size := 5)),
        pd.to_datetime("2000-01-01"): rng.normal(size=size),
        pd.to_datetime("2000-01-02"): rng.normal(size=size),
        pd.to_datetime("2000-01-03"): rng.normal(size=size),
    },
    index=rng.choice([*ascii_lowercase], size=(size, 4)).view("<U4").ravel(),
)
df


,sector,2000-01-01 00:00:00,2000-01-02 00:00:00,2000-01-03 00:00:00
yyec,energy,0.052029,-1.320431,0.188519
dszy,tech,0.683686,-0.661528,-0.633194
rzwa,healthcare,1.003962,0.935050,-0.377564
dwcz,healthcare,-0.617907,0.049055,-1.091146
vyjd,tech,1.822011,2.002393,-1.277680


In [ ]:
# Melt: take part of my columns. (in this case everything but sector)
# Move it down into the data
df.melt(value_vars=df.columns.difference(["sector"]), var_name="date")


,date,value
0,2000-01-01,0.781311
1,2000-01-01,0.264456
2,2000-01-01,-0.313923
3,2000-01-01,1.458021
4,2000-01-01,1.960258
5,2000-01-02,1.801635
6,2000-01-02,1.315104
7,2000-01-02,0.357380
8,2000-01-02,-1.208319
9,2000-01-02,-0.004454


In [ ]:
# Melt: don't ignore index to keep the labels
df.melt(
    value_vars=df.columns.difference(["sector"]), ignore_index=False, var_name="date"
)


,date,value
psft,2000-01-01,0.781311
bokz,2000-01-01,0.264456
fycq,2000-01-01,-0.313923
pxhx,2000-01-01,1.458021
rxft,2000-01-01,1.960258
psft,2000-01-02,1.801635
bokz,2000-01-02,1.315104
fycq,2000-01-02,0.357380
pxhx,2000-01-02,-1.208319
rxft,2000-01-02,-0.004454


In [ ]:
# Melt: set id_vars to duplicate the variables that are supposed to be included as labels
# Each date each sector each value
df.melt(
    id_vars=["sector"],
    value_vars=df.columns.difference(["sector"]),
    ignore_index=False,
    var_name="date",
)


,sector,date,value
psft,energy,2000-01-01,0.781311
bokz,healthcare,2000-01-01,0.264456
fycq,tech,2000-01-01,-0.313923
pxhx,tech,2000-01-01,1.458021
rxft,energy,2000-01-01,1.960258
psft,energy,2000-01-02,1.801635
bokz,healthcare,2000-01-02,1.315104
fycq,tech,2000-01-02,0.357380
pxhx,tech,2000-01-02,-1.208319
rxft,energy,2000-01-02,-0.004454


In [ ]:
# Would probably want Date as the index to pick dates / date ranges
df.melt(
    id_vars=["sector"],
    value_vars=df.columns.difference(["sector"]),
    ignore_index=False,
    var_name="date",
).set_index("date", append=True).sort_index()


sector     value
     date                            
bokz 2000-01-01  healthcare  0.264456
     2000-01-02  healthcare  1.315104
     2000-01-03  healthcare -1.288361
fycq 2000-01-01        tech -0.313923
     2000-01-02        tech  0.357380
     2000-01-03        tech  0.395122
psft 2000-01-01      energy  0.781311
     2000-01-02      energy  1.801635
     2000-01-03      energy  0.656475
pxhx 2000-01-01        tech  1.458021
     2000-01-02        tech -1.208319
     2000-01-03        tech  0.429864
rxft 2000-01-01      energy  1.960258
     2000-01-02      energy -0.004454
     2000-01-03      energy  0.696043

In [ ]:
# Clean it up by swapping level
df.melt(
    id_vars=["sector"],
    value_vars=df.columns.difference(["sector"]),
    ignore_index=False,
    var_name="date",
).set_index("date", append=True).sort_index().swaplevel().sort_index()


sector     value
date                                 
2000-01-01 bokz  healthcare  0.264456
           fycq        tech -0.313923
           psft      energy  0.781311
           pxhx        tech  1.458021
           rxft      energy  1.960258
2000-01-02 bokz  healthcare  1.315104
           fycq        tech  0.357380
           psft      energy  1.801635
           pxhx        tech -1.208319
           rxft      energy -0.004454
2000-01-03 bokz  healthcare -1.288361
           fycq        tech  0.395122
           psft      energy  0.656475
           pxhx        tech  0.429864
           rxft      energy  0.696043

In [ ]:
# Back to stock tickers
tickers = rng.choice([*ascii_lowercase], size=(100, 4)).view("<U4").ravel()
dates = pd.date_range("2000-01-01", "2000-12-31", name="date")
prices = (
    100
    * rng.random(size=len(tickers))
    * rng.normal(loc=1, scale=0.01, size=(len(dates), len(tickers))).cumprod(axis=0)
).ravel()
volumes = rng.integers(-50_000, 50_000, size=len(dates) * len(tickers)).round(-2)

df = (
    pd.DataFrame(
        {
            "date": np.repeat(dates, len(tickers)),
            "ticker": np.tile(tickers, len(dates)),
            "price": prices,
            "volume": volumes,
        }
    )
    .set_index(["date", "ticker"])
    .sort_index()
)

df


price  volume
date       ticker                   
2000-01-01 aizh    19.870206  -32900
           akaw     8.256200   45200
           aliz    46.894627    3700
           alql    17.844492   18500
           altn     7.201043   19000
...                      ...     ...
2000-12-31 ydvk    48.471328  -26700
           ysqy    22.832941    6400
           zcvs    27.201878   49300
           zgod    71.762939    2700
           ztqw    23.852593  -33100

[36600 rows x 2 columns]

In [ ]:
# Pivot like dual of melt
# Just about a groupby and then unstack in one step

# get_level_values just picks the dates, not the tickers
df.groupby([df.index.get_level_values("date").to_period("M"), "ticker"]).sum()


price  volume
date    ticker                     
2000-01 aizh     624.046583  172700
        akaw     250.490247  -26200
        aliz    1475.988152  -28000
        alql     557.387588  383400
        altn     217.216816   17500
...                     ...     ...
2000-12 ydvk    1596.525998 -137500
        ysqy     679.873476 -150300
        zcvs     845.755135  308500
        zgod    2234.636820 -215200
        ztqw     720.602011 -112700

[1200 rows x 2 columns]

In [ ]:
# innermost layer of row index (ticker) now in columns
df.groupby([df.index.get_level_values("date").to_period("M"), "ticker"]).sum().unstack()


price                                                   \
ticker         aizh        akaw         aliz        alql        altn   
date                                                                   
2000-01  624.046583  250.490247  1475.988152  557.387588  217.216816   
2000-02  622.544586  238.973151  1341.352103  492.005010  194.052585   
2000-03  693.070816  267.342646  1300.732800  524.346038  198.338075   
2000-04  653.923669  286.200665  1203.136301  519.304214  180.375693   
2000-05  689.482933  318.562887  1287.645557  551.713101  176.117597   
2000-06  650.965670  304.630826  1284.196915  537.749716  159.898955   
2000-07  677.425714  316.261466  1360.423669  576.894593  159.747880   
2000-08  693.938895  327.277288  1321.680953  600.923023  163.769492   
2000-09  670.839857  327.752368  1241.505663  600.433923  164.612025   
2000-10  720.440749  330.436677  1264.588518  640.922631  175.660665   
2000-11  677.957596  312.571014  1270.965788  655.397274  174.043720   
2000-12  658.948554  330.911498  1298.391386  621.360699  178.795490   

                                                                        ...  \
ticker         aola         azbv         begr         behw        bolh  ...   
date                                                                    ...   
2000-01  744.417174  1193.774499  1477.661462  2776.125543  423.558573  ...   
2000-02  688.315588  1171.677378  1399.893866  2754.346112  370.035306  ...   
2000-03  773.323969  1282.458645  1449.133958  2912.016142  382.769231  ...   
2000-04  783.379083  1283.590008  1470.346919  2882.207978  374.940550  ...   
2000-05  769.837099  1314.877199  1551.786578  3026.294806  380.456767  ...   
2000-06  766.871615  1306.374242  1576.281605  3013.007978  361.343292  ...   
2000-07  840.086828  1303.676395  1654.883471  3164.466394  366.086325  ...   
2000-08  801.280924  1262.443592  1653.311860  3106.279920  356.125409  ...   
2000-09  786.492796  1181.327137  1611.782895  2852.945646  343.842640  ...   
2000-10  856.014249  1208.377484  1569.640971  3013.694451  357.009603  ...   
2000-11  822.011069  1240.037315  1658.809307  3028.019282  348.534118  ...   
2000-12  789.646323  1333.011312  1813.207806  3213.796227  358.766776  ...   

         volume                                                          \
ticker     wotv    wvca    xblm    xpwz    xxpo    ydvk    ysqy    zcvs   
date                                                                      
2000-01 -118200  -38100  139100   72100 -149700 -177100  370300 -130000   
2000-02   16900   10700  -75400 -377300  -28800  111500  250200 -178700   
2000-03   85300   77400 -190500  122500  -53400  -76200  141000 -232100   
2000-04   76700  -16300 -164000  200900 -169700   98100  148300  -72500   
2000-05 -248700 -267300 -203400 -433600   20400  -52100 -265900  -44300   
2000-06   20600  -76400 -183500 -112800  197100   11100  125400  -47600   
2000-07  327400  110200   36800   76100  -58400   14500 -152900   79900   
2000-08   98600  -11000  -18600  219700 -152800  478000  250200   87300   
2000-09  -95500  163200 -226900  124900   30900  -28400 -132700   15100   
2000-10    6400  -21200  283100    7000   65200  -27900  499500  -45400   
2000-11   78300  128500 -179500   45700 -262900  -92200  273400  -93400   
2000-12    1100  121400  113800  -83200  -11500 -137500 -150300  308500   

                         
ticker     zgod    ztqw  
date                     
2000-01  -33900 -181200  
2000-02   71600   39800  
2000-03   -4000  109700  
2000-04   54100  143300  
2000-05  110800  158200  
2000-06  -27500 -147200  
2000-07 -290800  174400  
2000-08  -63100  160800  
2000-09  -84600   45200  
2000-10  267500  -24500  
2000-11 -118500 -216400  
2000-12 -215200 -112700  

[12 rows x 200 columns]

In [ ]:
# Pick our new index, what we're adding to columns, and the aggregation function
df.pivot_table(
    index=df.index.get_level_values("date").to_period("M"),
    columns="ticker",
    aggfunc="sum",
)


price                                                   \
ticker         aizh        akaw         aliz        alql        altn   
date                                                                   
2000-01  624.046583  250.490247  1475.988152  557.387588  217.216816   
2000-02  622.544586  238.973151  1341.352103  492.005010  194.052585   
2000-03  693.070816  267.342646  1300.732800  524.346038  198.338075   
2000-04  653.923669  286.200665  1203.136301  519.304214  180.375693   
2000-05  689.482933  318.562887  1287.645557  551.713101  176.117597   
2000-06  650.965670  304.630826  1284.196915  537.749716  159.898955   
2000-07  677.425714  316.261466  1360.423669  576.894593  159.747880   
2000-08  693.938895  327.277288  1321.680953  600.923023  163.769492   
2000-09  670.839857  327.752368  1241.505663  600.433923  164.612025   
2000-10  720.440749  330.436677  1264.588518  640.922631  175.660665   
2000-11  677.957596  312.571014  1270.965788  655.397274  174.043720   
2000-12  658.948554  330.911498  1298.391386  621.360699  178.795490   

                                                                        ...  \
ticker         aola         azbv         begr         behw        bolh  ...   
date                                                                    ...   
2000-01  744.417174  1193.774499  1477.661462  2776.125543  423.558573  ...   
2000-02  688.315588  1171.677378  1399.893866  2754.346112  370.035306  ...   
2000-03  773.323969  1282.458645  1449.133958  2912.016142  382.769231  ...   
2000-04  783.379083  1283.590008  1470.346919  2882.207978  374.940550  ...   
2000-05  769.837099  1314.877199  1551.786578  3026.294806  380.456767  ...   
2000-06  766.871615  1306.374242  1576.281605  3013.007978  361.343292  ...   
2000-07  840.086828  1303.676395  1654.883471  3164.466394  366.086325  ...   
2000-08  801.280924  1262.443592  1653.311860  3106.279920  356.125409  ...   
2000-09  786.492796  1181.327137  1611.782895  2852.945646  343.842640  ...   
2000-10  856.014249  1208.377484  1569.640971  3013.694451  357.009603  ...   
2000-11  822.011069  1240.037315  1658.809307  3028.019282  348.534118  ...   
2000-12  789.646323  1333.011312  1813.207806  3213.796227  358.766776  ...   

         volume                                                          \
ticker     wotv    wvca    xblm    xpwz    xxpo    ydvk    ysqy    zcvs   
date                                                                      
2000-01 -118200  -38100  139100   72100 -149700 -177100  370300 -130000   
2000-02   16900   10700  -75400 -377300  -28800  111500  250200 -178700   
2000-03   85300   77400 -190500  122500  -53400  -76200  141000 -232100   
2000-04   76700  -16300 -164000  200900 -169700   98100  148300  -72500   
2000-05 -248700 -267300 -203400 -433600   20400  -52100 -265900  -44300   
2000-06   20600  -76400 -183500 -112800  197100   11100  125400  -47600   
2000-07  327400  110200   36800   76100  -58400   14500 -152900   79900   
2000-08   98600  -11000  -18600  219700 -152800  478000  250200   87300   
2000-09  -95500  163200 -226900  124900   30900  -28400 -132700   15100   
2000-10    6400  -21200  283100    7000   65200  -27900  499500  -45400   
2000-11   78300  128500 -179500   45700 -262900  -92200  273400  -93400   
2000-12    1100  121400  113800  -83200  -11500 -137500 -150300  308500   

                         
ticker     zgod    ztqw  
date                     
2000-01  -33900 -181200  
2000-02   71600   39800  
2000-03   -4000  109700  
2000-04   54100  143300  
2000-05  110800  158200  
2000-06  -27500 -147200  
2000-07 -290800  174400  
2000-08  -63100  160800  
2000-09  -84600   45200  
2000-10  267500  -24500  
2000-11 -118500 -216400  
2000-12 -215200 -112700  

[12 rows x 200 columns]

In [ ]:
# Makes no sense in this scenario
df.pivot(index="price", columns="volume")


price
1.157202
1.159316
1.161514
1.162548
1.163602
...
140.670605
140.973300
141.680706
142.516953


In [ ]:
# Sorts the index
df.sort_index()


price  volume
date       ticker                   
2000-01-01 aizh    19.870206  -32900
           akaw     8.256200   45200
           aliz    46.894627    3700
           alql    17.844492   18500
           altn     7.201043   19000
...                      ...     ...
2000-12-31 ydvk    48.471328  -26700
           ysqy    22.832941    6400
           zcvs    27.201878   49300
           zgod    71.762939    2700
           ztqw    23.852593  -33100

[36600 rows x 2 columns]

In [ ]:
# Can sort columns because that's an Index. ascending=False means descending
df.sort_index(axis="columns", ascending=False)


volume      price
date       ticker                   
2000-01-01 aizh    -32900  19.870206
           akaw     45200   8.256200
           aliz      3700  46.894627
           alql     18500  17.844492
           altn     19000   7.201043
...                   ...        ...
2000-12-31 ydvk    -26700  48.471328
           ysqy      6400  22.832941
           zcvs     49300  27.201878
           zgod      2700  71.762939
           ztqw    -33100  23.852593

[36600 rows x 2 columns]

In [ ]:
# Not involved in index to start with
df.sort_values("volume")


,,price,volume
date,ticker,,
2000-05-23,alql,17.977217,-50000
2000-09-12,hpzd,95.444294,-50000
2000-08-18,dyrb,3.756751,-50000
2000-02-04,bxyb,44.584685,-50000
2000-03-18,cjcc,67.041632,-50000
...,...,...,...
2000-10-17,skko,12.923974,50000
2000-01-30,altn,6.936653,50000
2000-02-11,qmvv,44.025327,50000


In [ ]:
# Pop off the current index name
df.reset_index("ticker")


,ticker,price,volume
date,,,
2000-01-01,aizh,19.870206,-32900
2000-01-01,akaw,8.256200,45200
2000-01-01,aliz,46.894627,3700
2000-01-01,alql,17.844492,18500
2000-01-01,altn,7.201043,19000
...,...,...,...
2000-12-31,ydvk,48.471328,-26700
2000-12-31,ysqy,22.832941,6400
2000-12-31,zcvs,27.201878,49300


In [ ]:
# Or pop it off and just delete it
df.reset_index("ticker", drop=True)


,price,volume
date,,
2000-01-01,19.870206,-32900
2000-01-01,8.256200,45200
2000-01-01,46.894627,3700
2000-01-01,17.844492,18500
2000-01-01,7.201043,19000
...,...,...
2000-12-31,48.471328,-26700
2000-12-31,22.832941,6400
2000-12-31,27.201878,49300


In [ ]:
# Set Index moves from column index to row index
df.reset_index("ticker").set_index("ticker")


,price,volume
ticker,,
aizh,19.870206,-32900
akaw,8.256200,45200
aliz,46.894627,3700
alql,17.844492,18500
altn,7.201043,19000
...,...,...
ydvk,48.471328,-26700
ysqy,22.832941,6400
zcvs,27.201878,49300


In [ ]:
# Can be appended to play nicely
df.reset_index("ticker").set_index("ticker", append=True)


price  volume
date       ticker                   
2000-01-01 aizh    19.870206  -32900
           akaw     8.256200   45200
           aliz    46.894627    3700
           alql    17.844492   18500
           altn     7.201043   19000
...                      ...     ...
2000-12-31 ydvk    48.471328  -26700
           ysqy    22.832941    6400
           zcvs    27.201878   49300
           zgod    71.762939    2700
           ztqw    23.852593  -33100

[36600 rows x 2 columns]

In [ ]:
# Similar to resetting index with drop
# Take off part of the index and drop by name
df.droplevel("ticker")


,price,volume
date,,
2000-01-01,19.870206,-32900
2000-01-01,8.256200,45200
2000-01-01,46.894627,3700
2000-01-01,17.844492,18500
2000-01-01,7.201043,19000
...,...,...
2000-12-31,48.471328,-26700
2000-12-31,22.832941,6400
2000-12-31,27.201878,49300


In [ ]:
# Alter the dataset, don't wipe the index, just set it to a portion of the index with the values from original data
df.reindex(pd.date_range("2000-02-01", periods=4, freq="14D"), level="date")


price  volume
date       ticker                   
2000-02-01 aizh    21.321908   39100
           akaw     8.025052   42200
           aliz    46.122748  -18200
           alql    17.510511   29300
           altn     6.860670   18300
...                      ...     ...
2000-03-14 ydvk    63.802320  -37200
           ysqy    16.814420  -49300
           zcvs    34.492483       0
           zgod    59.370338  -42300
           ztqw    22.437787    6100

[400 rows x 2 columns]

In [ ]:
# About setting the values of the index, given the actual new values
df.set_axis(df.columns.str.title(), axis="columns")


Price  Volume
date       ticker                   
2000-01-01 aizh    19.870206  -32900
           akaw     8.256200   45200
           aliz    46.894627    3700
           alql    17.844492   18500
           altn     7.201043   19000
...                      ...     ...
2000-12-31 ydvk    48.471328  -26700
           ysqy    22.832941    6400
           zcvs    27.201878   49300
           zgod    71.762939    2700
           ztqw    23.852593  -33100

[36600 rows x 2 columns]

In [ ]:
# similar to set_axis but with a mapper function
df.rename(str.title, axis="columns")


Price  Volume
date       ticker                   
2000-01-01 aizh    19.870206  -32900
           akaw     8.256200   45200
           aliz    46.894627    3700
           alql    17.844492   18500
           altn     7.201043   19000
...                      ...     ...
2000-12-31 ydvk    48.471328  -26700
           ysqy    22.832941    6400
           zcvs    27.201878   49300
           zgod    71.762939    2700
           ztqw    23.852593  -33100

[36600 rows x 2 columns]

In [ ]:
# uniform transformation on the dates. pulled back one day
df.rename(lambda dt: dt - timedelta(days=1), level="date")


price  volume
date       ticker                   
1999-12-31 aizh    19.870206  -32900
           akaw     8.256200   45200
           aliz    46.894627    3700
           alql    17.844492   18500
           altn     7.201043   19000
...                      ...     ...
2000-12-30 ydvk    48.471328  -26700
           ysqy    22.832941    6400
           zcvs    27.201878   49300
           zgod    71.762939    2700
           ztqw    23.852593  -33100

[36600 rows x 2 columns]

In [ ]:
# switches order of levels in index
df.swaplevel()


,,price,volume
ticker,date,,
aizh,2000-01-01,19.870206,-32900
akaw,2000-01-01,8.256200,45200
aliz,2000-01-01,46.894627,3700
alql,2000-01-01,17.844492,18500
altn,2000-01-01,7.201043,19000
...,...,...,...
ydvk,2000-12-31,48.471328,-26700
ysqy,2000-12-31,22.832941,6400
zcvs,2000-12-31,27.201878,49300


In [ ]:
# Explicitly set the new ordering for index levels
df.reorder_levels(["ticker", "date"])


,,price,volume
ticker,date,,
aizh,2000-01-01,19.870206,-32900
akaw,2000-01-01,8.256200,45200
aliz,2000-01-01,46.894627,3700
alql,2000-01-01,17.844492,18500
altn,2000-01-01,7.201043,19000
...,...,...,...
ydvk,2000-12-31,48.471328,-26700
ysqy,2000-12-31,22.832941,6400
zcvs,2000-12-31,27.201878,49300


In [ ]:
# Axis helps with aligning for stack, unstacking, and multi-index index alignment
df.rename_axis(["value"], axis="columns")


value                  price  volume
date       ticker                   
2000-01-01 aizh    19.870206  -32900
           akaw     8.256200   45200
           aliz    46.894627    3700
           alql    17.844492   18500
           altn     7.201043   19000
...                      ...     ...
2000-12-31 ydvk    48.471328  -26700
           ysqy    22.832941    6400
           zcvs    27.201878   49300
           zgod    71.762939    2700
           ztqw    23.852593  -33100

[36600 rows x 2 columns]

In [ ]:
df.rename_axis(["value"], axis="columns").columns, df.columns


(Index(['price', 'volume'], dtype='object', name='value'),
 Index(['price', 'volume'], dtype='object'))

In [ ]:
# there's a None name on the stacking
df.stack().index


MultiIndex([('2000-01-01', 'aizh',  'price'),
            ('2000-01-01', 'aizh', 'volume'),
            ('2000-01-01', 'akaw',  'price'),
            ('2000-01-01', 'akaw', 'volume'),
            ('2000-01-01', 'aliz',  'price'),
            ('2000-01-01', 'aliz', 'volume'),
            ('2000-01-01', 'alql',  'price'),
            ('2000-01-01', 'alql', 'volume'),
            ('2000-01-01', 'altn',  'price'),
            ('2000-01-01', 'altn', 'volume'),
            ...
            ('2000-12-31', 'ydvk',  'price'),
            ('2000-12-31', 'ydvk', 'volume'),
            ('2000-12-31', 'ysqy',  'price'),
            ('2000-12-31', 'ysqy', 'volume'),
            ('2000-12-31', 'zcvs',  'price'),
            ('2000-12-31', 'zcvs', 'volume'),
            ('2000-12-31', 'zgod',  'price'),
            ('2000-12-31', 'zgod', 'volume'),
            ('2000-12-31', 'ztqw',  'price'),
            ('2000-12-31', 'ztqw', 'volume')],
           names=['date', 'ticker', None], length=73200)

In [ ]:
# But not if we name it first
df.rename_axis(["value"], axis="columns").stack().index


MultiIndex([('2000-01-01', 'aizh',  'price'),
            ('2000-01-01', 'aizh', 'volume'),
            ('2000-01-01', 'akaw',  'price'),
            ('2000-01-01', 'akaw', 'volume'),
            ('2000-01-01', 'aliz',  'price'),
            ('2000-01-01', 'aliz', 'volume'),
            ('2000-01-01', 'alql',  'price'),
            ('2000-01-01', 'alql', 'volume'),
            ('2000-01-01', 'altn',  'price'),
            ('2000-01-01', 'altn', 'volume'),
            ...
            ('2000-12-31', 'ydvk',  'price'),
            ('2000-12-31', 'ydvk', 'volume'),
            ('2000-12-31', 'ysqy',  'price'),
            ('2000-12-31', 'ysqy', 'volume'),
            ('2000-12-31', 'zcvs',  'price'),
            ('2000-12-31', 'zcvs', 'volume'),
            ('2000-12-31', 'zgod',  'price'),
            ('2000-12-31', 'zgod', 'volume'),
            ('2000-12-31', 'ztqw',  'price'),
            ('2000-12-31', 'ztqw', 'volume')],
           names=['date', 'ticker', 'value'], length=73200)

In [ ]:
# Like transpose
df.swapaxes("index", "columns")


date      2000-01-01                                                       \
ticker          aizh        akaw         aliz          alql          altn   
price      19.870206      8.2562    46.894627     17.844492      7.201043   
volume -32900.000000  45200.0000  3700.000000  18500.000000  19000.000000   

date                                                                          \
ticker          aola          azbv          begr          behw          bolh   
price      24.624905     37.603556     47.492132     88.204172     12.882422   
volume  12300.000000  33200.000000 -34500.000000  46700.000000  40400.000000   

date    ...    2000-12-31                                                      \
ticker  ...          wotv         wvca         xblm         xpwz         xxpo   
price   ...     61.009833    11.148302    63.562751     13.21313    85.887365   
volume  ...  22800.000000 -1400.000000  4100.000000 -14400.00000 -7000.000000   

date                                                                        
ticker          ydvk         ysqy          zcvs         zgod          ztqw  
price      48.471328    22.832941     27.201878    71.762939     23.852593  
volume -26700.000000  6400.000000  49300.000000  2700.000000 -33100.000000  

[2 rows x 36600 columns]

In [ ]:
df.T


date      2000-01-01                                                       \
ticker          aizh        akaw         aliz          alql          altn   
price      19.870206      8.2562    46.894627     17.844492      7.201043   
volume -32900.000000  45200.0000  3700.000000  18500.000000  19000.000000   

date                                                                          \
ticker          aola          azbv          begr          behw          bolh   
price      24.624905     37.603556     47.492132     88.204172     12.882422   
volume  12300.000000  33200.000000 -34500.000000  46700.000000  40400.000000   

date    ...    2000-12-31                                                      \
ticker  ...          wotv         wvca         xblm         xpwz         xxpo   
price   ...     61.009833    11.148302    63.562751     13.21313    85.887365   
volume  ...  22800.000000 -1400.000000  4100.000000 -14400.00000 -7000.000000   

date                                                                        
ticker          ydvk         ysqy          zcvs         zgod          ztqw  
price      48.471328    22.832941     27.201878    71.762939     23.852593  
volume -26700.000000  6400.000000  49300.000000  2700.000000 -33100.000000  

[2 rows x 36600 columns]

In [ ]:
# Take something that could be a series and turn it into one
# Still a df
df.squeeze()


price  volume
date       ticker                   
2000-01-01 aizh    19.870206  -32900
           akaw     8.256200   45200
           aliz    46.894627    3700
           alql    17.844492   18500
           altn     7.201043   19000
...                      ...     ...
2000-12-31 ydvk    48.471328  -26700
           ysqy    22.832941    6400
           zcvs    27.201878   49300
           zgod    71.762939    2700
           ztqw    23.852593  -33100

[36600 rows x 2 columns]

In [ ]:
# Reduced to Series
df[["price"]].squeeze()


date        ticker
2000-01-01  aizh      19.870206
            akaw       8.256200
            aliz      46.894627
            alql      17.844492
            altn       7.201043
                        ...    
2000-12-31  ydvk      48.471328
            ysqy      22.832941
            zcvs      27.201878
            zgod      71.762939
            ztqw      23.852593
Name: price, Length: 36600, dtype: float64

In [ ]:
# Can squeeze on index as well
df.loc["2000-01-01"].squeeze(axis="index")


,price,volume
ticker,,
aizh,19.870206,-32900
akaw,8.256200,45200
aliz,46.894627,3700
alql,17.844492,18500
altn,7.201043,19000
...,...,...
ydvk,56.259797,-39200
ysqy,17.719073,23400
zcvs,35.152654,5200


In [ ]:
s = pd.Series([*rng.integers(10, size=(3, 3))])
s


0    [4, 3, 1]
1    [3, 6, 6]
2    [5, 1, 1]
dtype: object

In [ ]:
# Explode expands all nested entries into additional rows
s.explode()


0    4
0    3
0    1
1    3
1    6
1    6
2    5
2    1
2    1
dtype: object

In [ ]:
# About accessing one value using labels
df.at[("2000-01-01", "aizh"), "price"]


19.870205771343183

In [ ]:
# Or one value using raw memory location
df.iat[0, 0]


19.870205771343183

In [ ]:
# Just as loc selects by labels
df.loc[("2000-01-01", "aizh"), "price"]


19.870205771343183

In [ ]:
# And iloc uses memory indices
df.iloc[0, 0]


19.870205771343183

In [ ]:
# Basically convenience for iloc first few entries
df.head(3)


price  volume
date       ticker                   
2000-01-01 aizh    19.870206  -32900
           akaw     8.256200   45200
           aliz    46.894627    3700

In [ ]:
df.iloc[:3]


price  volume
date       ticker                   
2000-01-01 aizh    19.870206  -32900
           akaw     8.256200   45200
           aliz    46.894627    3700

In [ ]:
# Basically iloc for last few entries
df.tail(3)


price  volume
date       ticker                   
2000-12-31 zcvs    27.201878   49300
           zgod    71.762939    2700
           ztqw    23.852593  -33100

In [ ]:
df.iloc[-3:]


price  volume
date       ticker                   
2000-12-31 zcvs    27.201878   49300
           zgod    71.762939    2700
           ztqw    23.852593  -33100

In [ ]:
# Take a random iloc sampling of 3 values
df.sample(3)


,,price,volume
date,ticker,,
2000-02-14,gzbu,10.933629,2100
2000-08-14,nwqw,54.755637,-19500
2000-07-23,gqnh,40.029220,47200


In [ ]:
df.iloc[rng.choice(len(df), size=3)]


,,price,volume
date,ticker,,
2000-06-28,xxpo,90.543372,-38800
2000-12-16,hcnz,7.249015,-19100
2000-11-05,verk,72.215582,-32700


In [ ]:
# About adding values as matched up by index
df.add(df)


price  volume
date       ticker                    
2000-01-01 aizh     39.740412  -65800
           akaw     16.512401   90400
           aliz     93.789254    7400
           alql     35.688985   37000
           altn     14.402086   38000
...                       ...     ...
2000-12-31 ydvk     96.942656  -53400
           ysqy     45.665882   12800
           zcvs     54.403756   98600
           zgod    143.525877    5400
           ztqw     47.705185  -66200

[36600 rows x 2 columns]

In [ ]:
# Subtract
df.sub(df)


price  volume
date       ticker               
2000-01-01 aizh      0.0       0
           akaw      0.0       0
           aliz      0.0       0
           alql      0.0       0
           altn      0.0       0
...                  ...     ...
2000-12-31 ydvk      0.0       0
           ysqy      0.0       0
           zcvs      0.0       0
           zgod      0.0       0
           ztqw      0.0       0

[36600 rows x 2 columns]

In [ ]:
# Not so shorthand
df.subtract(df)


price  volume
date       ticker               
2000-01-01 aizh      0.0       0
           akaw      0.0       0
           aliz      0.0       0
           alql      0.0       0
           altn      0.0       0
...                  ...     ...
2000-12-31 ydvk      0.0       0
           ysqy      0.0       0
           zcvs      0.0       0
           zgod      0.0       0
           ztqw      0.0       0

[36600 rows x 2 columns]

In [ ]:
# Multiplication
df.mul(df)


price      volume
date       ticker                         
2000-01-01 aizh     394.825077  1082410000
           akaw      68.164846  2043040000
           aliz    2199.106037    13690000
           alql     318.425909   342250000
           altn      51.855021   361000000
...                        ...         ...
2000-12-31 ydvk    2349.469647   712890000
           ysqy     521.343197    40960000
           zcvs     739.942178  2430490000
           zgod    5149.919378     7290000
           ztqw     568.946177  1095610000

[36600 rows x 2 columns]

In [ ]:
df.multiply(df)


price      volume
date       ticker                         
2000-01-01 aizh     394.825077  1082410000
           akaw      68.164846  2043040000
           aliz    2199.106037    13690000
           alql     318.425909   342250000
           altn      51.855021   361000000
...                        ...         ...
2000-12-31 ydvk    2349.469647   712890000
           ysqy     521.343197    40960000
           zcvs     739.942178  2430490000
           zgod    5149.919378     7290000
           ztqw     568.946177  1095610000

[36600 rows x 2 columns]

In [ ]:
# Division that might get messed up by floating point arithmetic
df.div(df)
df.divide(df)


price  volume
date       ticker               
2000-01-01 aizh      1.0     1.0
           akaw      1.0     1.0
           aliz      1.0     1.0
           alql      1.0     1.0
           altn      1.0     1.0
...                  ...     ...
2000-12-31 ydvk      1.0     1.0
           ysqy      1.0     1.0
           zcvs      1.0     1.0
           zgod      1.0     1.0
           ztqw      1.0     1.0

[36600 rows x 2 columns]

In [ ]:
# Explicit floor division (df // df)
df.floordiv(df * 1.3)


price  volume
date       ticker               
2000-01-01 aizh      0.0     0.0
           akaw      0.0     0.0
           aliz      0.0     0.0
           alql      0.0     0.0
           altn      0.0     0.0
...                  ...     ...
2000-12-31 ydvk      0.0     0.0
           ysqy      0.0     0.0
           zcvs      0.0     0.0
           zgod      0.0     0.0
           ztqw      0.0     0.0

[36600 rows x 2 columns]

In [ ]:
# Or use the types of the entries (df / df)
df.truediv(df * 1.3)


price    volume
date       ticker                    
2000-01-01 aizh    0.769231  0.769231
           akaw    0.769231  0.769231
           aliz    0.769231  0.769231
           alql    0.769231  0.769231
           altn    0.769231  0.769231
...                     ...       ...
2000-12-31 ydvk    0.769231  0.769231
           ysqy    0.769231  0.769231
           zcvs    0.769231  0.769231
           zgod    0.769231  0.769231
           ztqw    0.769231  0.769231

[36600 rows x 2 columns]

In [ ]:
# If we had 2 columns, want to align series rows against df columns. df ** s
df.pow(pd.Series({"price": 0.5, "volume": 2}))


price  volume
date       ticker                   
2000-01-01 aizh    19.870206  -32900
           akaw     8.256200   45200
           aliz    46.894627    3700
           alql    17.844492   18500
           altn     7.201043   19000
...                      ...     ...
2000-12-31 ydvk    48.471328  -26700
           ysqy    22.832941    6400
           zcvs    27.201878   49300
           zgod    71.762939    2700
           ztqw    23.852593  -33100

[36600 rows x 2 columns]

In [ ]:
# Same for mod. df % s
df.mod(pd.Series({"price": 2, "volume": 3}))


price  volume
date       ticker                  
2000-01-01 aizh    1.870206       1
           akaw    0.256200       2
           aliz    0.894627       1
           alql    1.844492       2
           altn    1.201043       1
...                     ...     ...
2000-12-31 ydvk    0.471328       0
           ysqy    0.832941       1
           zcvs    1.201878       1
           zgod    1.762939       0
           ztqw    1.852593       2

[36600 rows x 2 columns]

In [ ]:
# Basically just Multiply and sum index aligned
df.dot(pd.Series({"price": 2, "volume": 3}))


date        ticker
2000-01-01  aizh      -98660.259588
            akaw      135616.512401
            aliz       11193.789254
            alql       55535.688985
            altn       57014.402086
                          ...      
2000-12-31  ydvk      -80003.057344
            ysqy       19245.665882
            zcvs      147954.403756
            zgod        8243.525877
            ztqw      -99252.294815
Length: 36600, dtype: float64

In [ ]:
df.mul(pd.Series({"price": 2, "volume": 3})).sum(axis="columns")


date        ticker
2000-01-01  aizh      -98660.259588
            akaw      135616.512401
            aliz       11193.789254
            alql       55535.688985
            altn       57014.402086
                          ...      
2000-12-31  ydvk      -80003.057344
            ysqy       19245.665882
            zcvs      147954.403756
            zgod        8243.525877
            ztqw      -99252.294815
Length: 36600, dtype: float64

In [ ]:
# comparisons are index aligned
df == df, df != df, df <= df, df >= df, df < df, df > df


(                   price  volume
 date       ticker               
 2000-01-01 aizh     True    True
            akaw     True    True
            aliz     True    True
            alql     True    True
            altn     True    True
 ...                  ...     ...
 2000-12-31 ydvk     True    True
            ysqy     True    True
            zcvs     True    True
            zgod     True    True
            ztqw     True    True
 
 [36600 rows x 2 columns],
                    price  volume
 date       ticker               
 2000-01-01 aizh    False   False
            akaw    False   False
            aliz    False   False
            alql    False   False
            altn    False   False
 ...                  ...     ...
 2000-12-31 ydvk    False   False
            ysqy    False   False
            zcvs    False   False
            zgod    False   False
            ztqw    False   False
 
 [36600 rows x 2 columns],
                    price  volume
 date       ticker      

In [ ]:
df1 = pd.DataFrame(
    {
        "a": rng.integers(-10, 10, size=(size := 3)),
        "b": rng.integers(-10, 10, size=size),
    },
    index=pd.date_range("2000-01-01", periods=size),
)
df2 = pd.DataFrame(
    {
        "b": rng.integers(-10, 10, size=size),
        "c": rng.integers(-10, 10, size=size),
    },
    index=pd.date_range("2000-01-02", periods=size),
)

df1, df2


(            a  b
 2000-01-01  6 -9
 2000-01-02 -8  4
 2000-01-03 -1  1,
             b  c
 2000-01-02  4 -6
 2000-01-03  9  1
 2000-01-04 -2  1)

In [ ]:
# Show how they would match up / align in an operation
# Both directions
for x in df1.align(df2):
    print(x)


              a    b   c
2000-01-01  6.0 -9.0 NaN
2000-01-02 -8.0  4.0 NaN
2000-01-03 -1.0  1.0 NaN
2000-01-04  NaN  NaN NaN
             a    b    c
2000-01-01 NaN  NaN  NaN
2000-01-02 NaN  4.0 -6.0
2000-01-03 NaN  9.0  1.0
2000-01-04 NaN -2.0  1.0


In [ ]:
# Show how they would match up / align in an operation specificly on index
# Both directions
for x in df1.align(df2, axis="index"):
    print(x)


              a    b
2000-01-01  6.0 -9.0
2000-01-02 -8.0  4.0
2000-01-03 -1.0  1.0
2000-01-04  NaN  NaN
              b    c
2000-01-01  NaN  NaN
2000-01-02  4.0 -6.0
2000-01-03  9.0  1.0
2000-01-04 -2.0  1.0


In [ ]:
# Take alignment and join the data as possible
df1.join(df2, lsuffix="1", rsuffix="2")


,a,b1,b2,c
2000-01-01,6,-9,NaN,NaN
2000-01-02,-8,4,4.0,-6.0
2000-01-03,-1,1,9.0,1.0


In [ ]:
# Control join like SQL
df1.join(df2, lsuffix="1", rsuffix="2", how="outer")


,a,b1,b2,c
2000-01-01,6.0,-9.0,NaN,NaN
2000-01-02,-8.0,4.0,4.0,-6.0
2000-01-03,-1.0,1.0,9.0,1.0
2000-01-04,NaN,NaN,-2.0,1.0


In [ ]:
# Merge is general
df.merge


<bound method DataFrame.merge of                        price  volume
date       ticker                   
2000-01-01 aizh    19.870206  -32900
           akaw     8.256200   45200
           aliz    46.894627    3700
           alql    17.844492   18500
           altn     7.201043   19000
...                      ...     ...
2000-12-31 ydvk    48.471328  -26700
           ysqy    22.832941    6400
           zcvs    27.201878   49300
           zgod    71.762939    2700
           ztqw    23.852593  -33100

[36600 rows x 2 columns]>

In [ ]:
# How Join could have worked instead of lsfufix and rsuffix
# First set both sides as having a new level of index left and right
df1.set_axis(pd.MultiIndex.from_product([["left"], df1.columns]), axis="columns").join
    df2.set_axis(pd.MultiIndex.from_product([["right"], df2.columns]), axis="columns")
)
axis='columns').join(df2.set_axis(pd.MultiIndex.from_product([['right'], df2.columns]), axis='columns'))

left    right     
              a  b     b    c
2000-01-01    6 -9   NaN  NaN
2000-01-02   -8  4   4.0 -6.0
2000-01-03   -1  1   9.0  1.0

In [ ]:
# Then do a rename
df1.set_axis(pd.MultiIndex.from_product([["left"], df1.columns]), axis="columns").join(
    df2.set_axis(pd.MultiIndex.from_product([["right"], df2.columns]), axis="columns")
).rename_axis(["side", "columns"], axis="columns")


side       left    right     
columns       a  b     b    c
2000-01-01    6 -9   NaN  NaN
2000-01-02   -8  4   4.0 -6.0
2000-01-03   -1  1   9.0  1.0

In [ ]:
# Then do something with merged results like groupby
df1.set_axis(pd.MultiIndex.from_product([["left"], df1.columns]), axis="columns").join(
    df2.set_axis(pd.MultiIndex.from_product([["right"], df2.columns]), axis="columns")
).rename_axis(["side", "columns"], axis="columns").groupby('columns', axis='columns').max()

columns,a,b,c
2000-01-01,6.0,-9.0,NaN
2000-01-02,-8.0,4.0,-6.0
2000-01-03,-1.0,9.0,1.0


In [ ]:
# Absolute value of every value. Not index aware
df.abs()

price  volume
date       ticker                   
2000-01-01 aizh    19.870206   32900
           akaw     8.256200   45200
           aliz    46.894627    3700
           alql    17.844492   18500
           altn     7.201043   19000
...                      ...     ...
2000-12-31 ydvk    48.471328   26700
           ysqy    22.832941    6400
           zcvs    27.201878   49300
           zgod    71.762939    2700
           ztqw    23.852593   33100

[36600 rows x 2 columns]

In [ ]:
# Clip every value in df. Not very index aware either
df.clip(10, 100)

price  volume
date       ticker                   
2000-01-01 aizh    19.870206      10
           akaw    10.000000     100
           aliz    46.894627     100
           alql    17.844492     100
           altn    10.000000     100
...                      ...     ...
2000-12-31 ydvk    48.471328      10
           ysqy    22.832941     100
           zcvs    27.201878     100
           zgod    71.762939     100
           ztqw    23.852593      10

[36600 rows x 2 columns]

In [ ]:
# min is index aware, needs to know location of the min value
df.min()

price         1.157202
volume   -50000.000000
dtype: float64

In [ ]:
# And lets us index differently
df.min(axis='columns')

date        ticker
2000-01-01  aizh     -32900.000000
            akaw          8.256200
            aliz         46.894627
            alql         17.844492
            altn          7.201043
                          ...     
2000-12-31  ydvk     -26700.000000
            ysqy         22.832941
            zcvs         27.201878
            zgod         71.762939
            ztqw     -33100.000000
Length: 36600, dtype: float64

In [ ]:
# Get original indexing of columns
df.min(axis='index')

price         1.157202
volume   -50000.000000
dtype: float64

In [ ]:
# Same with max
df.max()

price       142.657282
volume    50000.000000
dtype: float64

In [ ]:
# Same with sum
df.sum()

price     1.932210e+06
volume    4.354400e+06
dtype: float64

In [ ]:
# Same with prod
df.prod()

/home/gar/miniconda3/envs/py310/lib/python3.10/site-packages/numpy/core/_methods.py:52: RuntimeWarning: overflow encountered in reduce
  return umr_prod(a, axis, dtype, out, keepdims, initial, where)


price     inf
volume    0.0
dtype: float64

In [ ]:
# Same with product
df.product()

price     inf
volume    0.0
dtype: float64

In [ ]:
# Same with any
df.any()

price     True
volume    True
dtype: bool

In [ ]:
# Same with all
df.all()

price      True
volume    False
dtype: bool

In [ ]:
# Same with mean
df.mean()

price      52.792624
volume    118.972678
dtype: float64

In [ ]:
# Same with std
df.std()

price        31.015202
volume    28850.772666
dtype: float64

In [ ]:
# Same with var
df.var()

price     9.619427e+02
volume    8.323671e+08
dtype: float64

In [ ]:
# Same with kurt
df.kurt()

price    -0.970741
volume   -1.201066
dtype: float64

In [ ]:
# Same with kurtosis
df.kurtosis()

price    -0.970741
volume   -1.201066
dtype: float64

In [ ]:
# Same with skew
df.skew()

price     0.081868
volume   -0.002927
dtype: float64

In [ ]:
# Same with median
# Indexing of result is index of the axis Against which you operated. (Default Against rows)
df.median()

price      56.261449
volume    100.000000
dtype: float64

In [ ]:
# Different. Provides 0th most common, 1st most common, etc
df.mode()

,price,volume
0,1.157202,-21300.0
1,1.159316,NaN
2,1.161514,NaN
3,1.162548,NaN
4,1.163602,NaN
...,...,...
36595,140.670605,NaN
36596,140.973300,NaN
36597,141.680706,NaN
36598,142.516953,NaN


In [ ]:
# Nonsensical
df.mode(axis='columns')

0             1
date       ticker                            
2000-01-01 aizh   -32900.000000     19.870206
           akaw        8.256200  45200.000000
           aliz       46.894627   3700.000000
           alql       17.844492  18500.000000
           altn        7.201043  19000.000000
...                         ...           ...
2000-12-31 ydvk   -26700.000000     48.471328
           ysqy       22.832941   6400.000000
           zcvs       27.201878  49300.000000
           zgod       71.762939   2700.000000
           ztqw   -33100.000000     23.852593

[36600 rows x 2 columns]

In [ ]:
# Count values of an axis
df.count()

price     36600
volume    36600
dtype: int64

In [ ]:
# Unique value counts over an axis
df.nunique()

price     36600
volume     1001
dtype: int64

In [ ]:
# mean absolute deviation
# good for presentations
df.mad()

price        26.807526
volume    25004.810450
dtype: float64

In [ ]:
# counts total number of times each tuple of column pairs appears
df.value_counts()

price       volume
1.157202    -8300     1
70.203082   -31500    1
70.189042   -33900    1
70.193033    36800    1
70.193509    28300    1
                     ..
35.358476    10000    1
35.356230    34200    1
35.355303    49600    1
35.353910    40200    1
142.657282  -44000    1
Length: 36600, dtype: int64

In [ ]:
# Those tuple pairs are the keys to the index as expected
df.value_counts().index

MultiIndex([(1.1572019204572928,  -8300),
            ( 70.20308234356753, -31500),
            ( 70.18904158462593, -33900),
            ( 70.19303338244887,  36800),
            ( 70.19350936781895,  28300),
            ( 70.19363901096825,  38300),
            (  70.1946782640241, -45800),
            ( 70.19790150545167, -28700),
            ( 70.20520525770625,  20000),
            ( 70.18756106020861,  11400),
            ...
            ( 35.38130301688002, -45300),
            ( 35.37688566818373, -26900),
            ( 35.37633075806288, -28600),
            ( 35.36531581805279, -35300),
            ( 35.36419351893424, -34400),
            ( 35.35847635837517,  10000),
            ( 35.35623025194556,  34200),
            ( 35.35530339036038,  49600),
            (35.353910310998884,  40200),
            (142.65728234258566, -44000)],
           names=['price', 'volume'], length=36600)

In [ ]:
# Get all data for row with smallest price 
df.nsmallest(1, columns=['price'])

,,price,volume
date,ticker,,
2000-05-28,crbs,1.157202,-8300


In [ ]:
# Get all data for 3 rows with smallest prices
df.nsmallest(3, columns=['price'])

,,price,volume
date,ticker,,
2000-05-28,crbs,1.157202,-8300
2000-05-27,crbs,1.159316,-7500
2000-05-26,crbs,1.161514,46000


In [ ]:
# Largest works the same
df.nlargest(1, columns=['price'])

,,price,volume
date,ticker,,
2000-11-18,uejr,142.657282,-44000


In [ ]:
# Can use the index of the 3 smallest prices, then look up those rows, then select just volume
df.loc[df.nsmallest(3, columns=['price']).index]['volume']

date        ticker
2000-05-28  crbs      -8300
2000-05-27  crbs      -7500
2000-05-26  crbs      46000
Name: volume, dtype: int64

In [ ]:
# Redundant in this case
df.nsmallest(3, columns=['price'])['volume']

date        ticker
2000-05-28  crbs      -8300
2000-05-27  crbs      -7500
2000-05-26  crbs      46000
Name: volume, dtype: int64

In [ ]:
# Min value in the index, to be used to look up the data at that max
df.idxmin()

price     (2000-05-28 00:00:00, crbs)
volume    (2000-01-08 00:00:00, altn)
dtype: object

In [ ]:
# Max value / last in the index
df.idxmax()

price     (2000-11-18 00:00:00, uejr)
volume    (2000-01-30 00:00:00, altn)
dtype: object

In [ ]:
# Perform operations down the rows. Should be grouped in this case
df.cummin()

price  volume
date       ticker                   
2000-01-01 aizh    19.870206  -32900
           akaw     8.256200  -32900
           aliz     8.256200  -32900
           alql     8.256200  -32900
           altn     7.201043  -32900
...                      ...     ...
2000-12-31 ydvk     1.157202  -50000
           ysqy     1.157202  -50000
           zcvs     1.157202  -50000
           zgod     1.157202  -50000
           ztqw     1.157202  -50000

[36600 rows x 2 columns]

In [ ]:
# probably want to groupby ticker then transform
df.groupby('ticker').transform(lambda s: s.cummin())

price  volume
date       ticker                   
2000-01-01 aizh    19.870206  -32900
           akaw     8.256200   45200
           aliz    46.894627    3700
           alql    17.844492   18500
           altn     7.201043   19000
...                      ...     ...
2000-12-31 ydvk    48.453064  -49600
           ysqy    15.926809  -49800
           zcvs    26.301952  -49900
           zgod    52.908498  -50000
           ztqw    20.750132  -49900

[36600 rows x 2 columns]

In [ ]:
# Take max going down the rows
df.cummax()

price  volume
date       ticker                    
2000-01-01 aizh     19.870206  -32900
           akaw     19.870206   45200
           aliz     46.894627   45200
           alql     46.894627   45200
           altn     46.894627   45200
...                       ...     ...
2000-12-31 ydvk    142.657282   50000
           ysqy    142.657282   50000
           zcvs    142.657282   50000
           zgod    142.657282   50000
           ztqw    142.657282   50000

[36600 rows x 2 columns]

In [ ]:
# Sum all values going down
df.cumsum()

price   volume
date       ticker                       
2000-01-01 aizh    1.987021e+01   -32900
           akaw    2.812641e+01    12300
           aliz    7.502103e+01    16000
           alql    9.286553e+01    34500
           altn    1.000666e+02    53500
...                         ...      ...
2000-12-31 ydvk    1.932064e+06  4329100
           ysqy    1.932087e+06  4335500
           zcvs    1.932114e+06  4384800
           zgod    1.932186e+06  4387500
           ztqw    1.932210e+06  4354400

[36600 rows x 2 columns]

In [ ]:
# Multiply all values going down
df.cumprod()

/home/gar/miniconda3/envs/py310/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)


price               volume
date       ticker                                   
2000-01-01 aizh    1.987021e+01               -32900
           akaw    1.640524e+02          -1487080000
           aliz    7.693176e+03       -5502196000000
           alql    1.372808e+05  -101790626000000000
           altn    9.885651e+05  2886233739502919680
...                         ...                  ...
2000-12-31 ydvk             inf                    0
           ysqy             inf                    0
           zcvs             inf                    0
           zgod             inf                    0
           ztqw             inf                    0

[36600 rows x 2 columns]

In [ ]:
# These are roughly equivalent to expanding
df.expanding().max(), df.cummax()

(                        price   volume
 date       ticker                     
 2000-01-01 aizh     19.870206 -32900.0
            akaw     19.870206  45200.0
            aliz     46.894627  45200.0
            alql     46.894627  45200.0
            altn     46.894627  45200.0
 ...                       ...      ...
 2000-12-31 ydvk    142.657282  50000.0
            ysqy    142.657282  50000.0
            zcvs    142.657282  50000.0
            zgod    142.657282  50000.0
            ztqw    142.657282  50000.0
 
 [36600 rows x 2 columns],
                         price  volume
 date       ticker                    
 2000-01-01 aizh     19.870206  -32900
            akaw     19.870206   45200
            aliz     46.894627   45200
            alql     46.894627   45200
            altn     46.894627   45200
 ...                       ...     ...
 2000-12-31 ydvk    142.657282   50000
            ysqy    142.657282   50000
            zcvs    142.657282   50000
            zgod    14

In [ ]:
df.expanding().min(), df.cummin()

(                       price   volume
 date       ticker                    
 2000-01-01 aizh    19.870206 -32900.0
            akaw     8.256200 -32900.0
            aliz     8.256200 -32900.0
            alql     8.256200 -32900.0
            altn     7.201043 -32900.0
 ...                      ...      ...
 2000-12-31 ydvk     1.157202 -50000.0
            ysqy     1.157202 -50000.0
            zcvs     1.157202 -50000.0
            zgod     1.157202 -50000.0
            ztqw     1.157202 -50000.0
 
 [36600 rows x 2 columns],
                        price  volume
 date       ticker                   
 2000-01-01 aizh    19.870206  -32900
            akaw     8.256200  -32900
            aliz     8.256200  -32900
            alql     8.256200  -32900
            altn     7.201043  -32900
 ...                      ...     ...
 2000-12-31 ydvk     1.157202  -50000
            ysqy     1.157202  -50000
            zcvs     1.157202  -50000
            zgod     1.157202  -50000
        

In [ ]:
df.expanding().sum(), df.cumsum()

(                          price     volume
 date       ticker                         
 2000-01-01 aizh    1.987021e+01   -32900.0
            akaw    2.812641e+01    12300.0
            aliz    7.502103e+01    16000.0
            alql    9.286553e+01    34500.0
            altn    1.000666e+02    53500.0
 ...                         ...        ...
 2000-12-31 ydvk    1.932064e+06  4329100.0
            ysqy    1.932087e+06  4335500.0
            zcvs    1.932114e+06  4384800.0
            zgod    1.932186e+06  4387500.0
            ztqw    1.932210e+06  4354400.0
 
 [36600 rows x 2 columns],
                           price   volume
 date       ticker                       
 2000-01-01 aizh    1.987021e+01   -32900
            akaw    2.812641e+01    12300
            aliz    7.502103e+01    16000
            alql    9.286553e+01    34500
            altn    1.000666e+02    53500
 ...                         ...      ...
 2000-12-31 ydvk    1.932064e+06  4329100
            ysqy    1

In [ ]:
# Not prod
df.expanding().prod()

AttributeError: 'Expanding' object has no attribute 'prod'

In [ ]:
# Expanding operations, like those before and ewm, preserve the index
df.ewm(alpha=.1).mean()

price        volume
date       ticker                         
2000-01-01 aizh    19.870206 -32900.000000
           akaw    13.757571   8205.263158
           aliz    25.985267   6542.804428
           alql    23.618075  10019.744112
           altn    19.609129  12212.671241
...                      ...           ...
2000-12-31 ydvk    53.616666  -1102.543010
           ysqy    50.538293   -352.288709
           zcvs    48.204652   4612.940162
           zgod    50.560481   4421.646146
           ztqw    47.889692    669.481531

[36600 rows x 2 columns]

In [ ]:
# Just move data, keep original index the same
df.shift(1)

price   volume
date       ticker                    
2000-01-01 aizh          NaN      NaN
           akaw    19.870206 -32900.0
           aliz     8.256200  45200.0
           alql    46.894627   3700.0
           altn    17.844492  18500.0
...                      ...      ...
2000-12-31 ydvk    85.887365  -7000.0
           ysqy    48.471328 -26700.0
           zcvs    22.832941   6400.0
           zgod    27.201878  49300.0
           ztqw    71.762939   2700.0

[36600 rows x 2 columns]

In [ ]:
# Do that shift and subtract at same time
df.diff(1)

price   volume
date       ticker                    
2000-01-01 aizh          NaN      NaN
           akaw   -11.614005  78100.0
           aliz    38.638426 -41500.0
           alql   -29.050135  14800.0
           altn   -10.643449    500.0
...                      ...      ...
2000-12-31 ydvk   -37.416037 -19700.0
           ysqy   -25.638387  33100.0
           zcvs     4.368937  42900.0
           zgod    44.561061 -46600.0
           ztqw   -47.910346 -35800.0

[36600 rows x 2 columns]

In [ ]:
# make a rolling mean by hand
df.rolling(3, min_periods=1).mean()

price        volume
date       ticker                         
2000-01-01 aizh    19.870206 -32900.000000
           akaw    14.063203   6150.000000
           aliz    25.007011   5333.333333
           alql    24.331773  22466.666667
           altn    23.980054  13733.333333
...                      ...           ...
2000-12-31 ydvk    49.190608 -16033.333333
           ysqy    52.397211  -9100.000000
           zcvs    32.835382   9666.666667
           zgod    40.599253  19466.666667
           ztqw    40.939137   6300.000000

[36600 rows x 2 columns]

In [ ]:
# groupby creates an indexing based on the groups
df.groupby('ticker').mean()

,price,volume
ticker,,
aizh,21.949687,-1942.349727
akaw,9.867242,-245.901639
aliz,42.761224,-1628.415301
alql,18.793546,-1240.710383
altn,5.854178,60.655738
...,...,...
ydvk,58.477315,332.786885
ysqy,19.404766,3706.284153
zcvs,31.102439,-965.027322


In [ ]:
# perate on series and change index of result to group data
df.groupby('ticker').agg(lambda s: s.mean())

,price,volume
ticker,,
aizh,21.949687,-1942.349727
akaw,9.867242,-245.901639
aliz,42.761224,-1628.415301
alql,18.793546,-1240.710383
altn,5.854178,60.655738
...,...,...
ydvk,58.477315,332.786885
ysqy,19.404766,3706.284153
zcvs,31.102439,-965.027322


In [ ]:
# Maintain original indexing
df.groupby('ticker').transform(lambda s: s.mean())

price       volume
date       ticker                        
2000-01-01 aizh    21.949687 -1942.349727
           akaw     9.867242  -245.901639
           aliz    42.761224 -1628.415301
           alql    18.793546 -1240.710383
           altn     5.854178    60.655738
...                      ...          ...
2000-12-31 ydvk    58.477315   332.786885
           ysqy    19.404766  3706.284153
           zcvs    31.102439  -965.027322
           zgod    64.635687  -911.475410
           ztqw    22.936185   408.196721

[36600 rows x 2 columns]

In [ ]:
# Then do a lookup on transformed data
df.groupby('ticker').transform(lambda s: s.mean()).loc[pd.IndexSlice[:, 'aizh', :]]

,price,volume
date,,
2000-01-01,21.949687,-1942.349727
2000-01-02,21.949687,-1942.349727
2000-01-03,21.949687,-1942.349727
2000-01-04,21.949687,-1942.349727
2000-01-05,21.949687,-1942.349727
...,...,...
2000-12-27,21.949687,-1942.349727
2000-12-28,21.949687,-1942.349727
2000-12-29,21.949687,-1942.349727


In [ ]:
# Breaks because it's not a datetime index
df.resample('M').mean()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'MultiIndex'

In [ ]:
# But it does if we're specific about what part of the index
df.resample('M', level='date').mean()

,price,volume
date,,
2000-01-31,51.539201,1191.290323
2000-02-29,51.900133,88.482759
2000-03-31,52.306717,379.064516
2000-04-30,52.608214,2.333333
2000-05-31,52.954118,-1513.000000
2000-06-30,53.125523,-134.100000
2000-07-31,53.105853,-461.354839
2000-08-31,53.059586,926.548387
2000-09-30,53.168439,470.566667


In [ ]:
# preserve original outer index by grouping first
df.groupby('ticker').apply(
    lambda s: s.resample('M', level='date').mean()
)

price       volume
ticker date                              
aizh   2000-01-31  20.130535  5570.967742
       2000-02-29  21.467055  2344.827586
       2000-03-31  22.357123   303.225806
       2000-04-30  21.797456   586.666667
       2000-05-31  22.241385 -7280.645161
...                      ...          ...
ztqw   2000-08-31  24.253022  5187.096774
       2000-09-30  24.472819  1506.666667
       2000-10-31  24.568214  -790.322581
       2000-11-30  24.578963 -7213.333333
       2000-12-31  23.245226 -3635.483871

[1200 rows x 2 columns]